### TOPIC CLASSIFICATION BENCHMARK 🔥

In [1]:
from transformers import pipeline 
import torch 
from rich import print, pretty, print_json


def get_device():
    device = "cuda" \
        if torch.cuda.is_available() \
        else "mps" if torch.backends.mps.is_available() \
        else "cpu"
    if device == "mps":
        import os
        os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
        os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"
    return torch.device(device)


In [2]:
device = get_device()


In [40]:
import dspy
from typing import List
from datasets import Dataset, DatasetDict
from dotenv import dotenv_values
from rich import print
secret = dotenv_values('../../.secret')
max_tokens=1024*3

gpt3  = dspy.OpenAI(
    model='gpt-3.5-turbo-0125',
    # model='gpt-4o',
    api_key=secret['OPEN_AI_API_KEY'],
    max_tokens=max_tokens,
    temperature=1.0
)

dspy.settings.configure(lm=gpt3)

# llama = dspy.GROQ(model='llama3-70b-8192', api_key=secret['GROQ_API_KEY'], max_tokens=max_tokens, temperature=1.0)
# dspy.settings.configure(lm=llama)


'dspy'

In [8]:

# greetings
instruct = """Generate a list of 300 english sentences about common chat message greetings.
Use emojis for some sentences, use a mix of casual language, formal language, slangs,  not provide names, should be impersonal, and provide a variety of examples. 
Do not repeat the examples. The length in words of the examples must be one of these categories: [4-7 words], [8-12 words], [13-16 words], [17-25 words].
The result should be a JSON following the format: greetings = { 'responses': ["string", "string", ....] }"""

# company_related
instruct = """Generate a list of 200 english sentences about the following text enclosed in triple backtick ```What kinds of questions might a person ask about the company during a WhatsApp, online, or chat interview?``` 
Do not repeat the examples. The examples must be different from the following: {cl}.
Use a mix of casual language, formal language, slangs,  not provide names, should be impersonal, use emojis for some examples, and provide a variety of examples. 
The result should be a JSON following the formar: company_related = { 'responses': [string, string, ....] }"""

# feedback
instruct = """Generate a list of 200 english sentences about the following text enclosed in triple backtick 
```In the context of an online process, flow, interview, or evaluation conducted via chat, what are the different ways a person might request information or an update about their current status comprehensively to increase their chances of securing the job?``` 
Do not repeat the examples. The examples must be different from the following: {fl}.
Use a mix of casual language, formal language, slangs, not provide names, should be impersonal, and provide a variety of examples. Some samples may include emojis.
The result should be a JSON following the formar: greetings = { 'responses': [string, string, ....] }
Do not repeat the sentences, each one on the list should be different."""


# later_continue
instruct = """Generate a list of 200 english sentences about the following text enclosed in triple backtick 
```In the context of an online process, flow, interview, or evaluation conducted via chat, what are the different ways a person: How might effectively respond in various scenarios where they need to indicate their intention to continue later, their inability to continue, their current time constraints, or any similar situations?``` 
Do not repeat the examples. The examples must be different from the following: {ll}.
Use a mix of casual language, formal language, slangs, not provide names, should be impersonal, and provide a variety of examples. Some samples may include emojis.
The result should be a JSON following the formar: later_continue = { 'responses': [string, string, ....] }
Do not repeat the sentences, each one on the list should be different."""


# not_continue
instruct = """Generate a list of 210 english sentences about the following text enclosed in triple backtick 
```In the context of an online process, flow, interview, or evaluation conducted via chat, what are the different ways a person: respond negatively if they do not want to continue, do not agree, or do not accept a process, flow, job, interview, or assessment``` 
Do not repeat the examples. The examples must be different from the following: {ncl}.
Use a mix of casual language, formal language, slangs, not provide names, should be impersonal, and provide a variety of examples. Some samples may include emojis.
The result should be a JSON following the formar: not_continue = { 'responses': [string, string, ....] }
Do not repeat the sentences, each one on the list should be different."""


# trouble
instruct = """Generate a list of 300 english sentences about the following text enclosed in triple backtick 
```In the context of an online process, flow, interview, or evaluation conducted via chat, what are the different ways a person: request for assistance, express difficulties, or voicing a complaint about the process``` 
Do not repeat the examples.
Use a mix of casual language, formal language, slangs, not provide names, should be impersonal, and provide a variety of examples. Some samples may include emojis.
The result should be a JSON following the formar: trouble = { 'responses': [string, string, ....] }
Do not repeat the sentences, each one on the list should be different."""


# himself #The sentence does not necessarily begin with "I am, I can, I..."
instruct = """Generate a list of 310 english sentences about the following text enclosed in triple backtick 
```In the context of call center and contact center industry, an online process , flow, interview, sales, customer services, or evaluation conducted via chat, 
what are the different ways a person talks about himself in the following topics: Technical Skills and Experience, Cultural Fit and Personality, Career Goals and Aspirations, behavioral questions.``` 
Do not repeat the examples. The sentence must begin with one of these two categories: "I am, I can, I..." or without "I am, I can, I..."
Use a mix of casual language slangs, a little formal language, not provide names, should be impersonal, and provide a variety of examples. Some samples may include emojis.
The result should be a JSON following the formar: himself = { 'responses': [string, string, ....] }
Do not repeat the sentences, each one on the list should be different."""


# continue
# instruct = """Generate a list of 200 english sentences about the following text enclosed in triple backtick 
# ```In the context of an online process, flow, interview, or evaluation conducted via chat, what are the different ways a person: respond affirmatively if they want to continue, agree, or accept a process, accept a flow, job, interview, or assessment``` 
# Do not repeat the examples. The examples must be different from the following: {ccl}.
# Use a mix of casual language, formal language, slangs, not provide names, should be impersonal, and provide a variety of examples. Some samples may include emojis.
# The result should be a JSON following the formar: continue = { 'responses': [string, string, ....] }
# Do not repeat the sentences, each one on the list should be different."""


# thanks
instruct = """Generate a list of 300 english sentences about the following text enclosed in triple backtick 
```In the context of call center and contact center industry, an online process , flow, interview, sales, customer services, or evaluation conducted via chat, 
What are the various ways in which people express gratitude or appreciation in different cultures and contexts?.``` 
Do not repeat the examples. Provide diversity.
Use a mix of casual language slangs, a little formal language, not provide names, should be impersonal, and provide a variety of examples. Some samples may include emojis.
The result should be a JSON following the formar: thanks = { 'responses': [string, string, ....] }
Do not repeat the sentences, each one on the list should be different."""


# instruct = """In the context of a conversation between interviewer and interviewee that may be conducted: by an online process, flow, interview, assessment, or evaluation conducted via chat, identify and 
# classify the 5 most prominent topics present in a variety of conversations. Consider different scenarios, provide diverse examples,
#  and account for a variety of people and personalities. Rank by most important.
#  The result should be a JSON following the formar:  ['topic 1', 'topic 2', ... 'topic N',] """



predict = dspy.Predict("question -> answer")
answer = predict(question=instruct).answer
print(answer)



```json
{
  "thanks": {
    "responses": [
      "Thank you!",
      "Much appreciated!",
      "Thanks a bunch!",
      "I really appreciate it!",
      "Thanks for your help!",
      "Thank you so much!",
      "I'm grateful for your assistance.",
      "Thanks a lot!",
      "Cheers!",
      "Thanks!",
      "Thanks a million!",
      "I'm forever grateful!",
      "Many thanks!",
      "Thanks for your support!",
      "I can't thank you enough!",
      "Thanks for everything!",
      "Thank you kindly!",
      "You're the best!",
      "Thanks again!",
      "Thank you, I appreciate it a lot!",
      "Grateful for your help!",
      "Big thanks!",
      "Thank you so very much!",
      "Thumbs up!",
      "Appreciate your time!",
      "Thanks, I owe you one!",
      "You're a lifesaver!",
      "Thank you, this means a lot!",
      "Thanks, you're awesome!",
      "Gracias!",
      "Thank you from the bottom of my heart!",
      "It means the world to me, thanks!",
      "Appreciate it!",
      "Thanks for the effort!",
      "Thanks a ton!",
      "Thanks in advance!",
      "Thank you for your kindness!",
      "Thank you for your cooperation!",
      "I'm grateful to you.",
      "Thank you for your patience!",
      "Much obliged!",
      "Your help was invaluable.",
      "I'm so thankful!",
      "Couldn't have done it without you!",
      "You've been a big help, thanks!",
      "Thank you for your understanding.",
      "I appreciate your assistance.",
      "I'd like to thank you sincerely.",
      "Thanks for the input!",
      "Thanks for being there!",
      "Thank you for your dedication!",
      "Thank you for the information!",
      "Thanks, that's very kind of you!",
      "Thankful for your guidance.",
      "Thanks for your quick response!",
      "Thank you for the update!",
      "You really helped me out, thanks!",
      "Thanks for your quick action!",
      "You have my gratitude.",
      "Thanks for being so helpful!",
      "Hats off to you!",
      "Thanks for the advice!",
      "Thank you for your service!",
      "You've been very supportive, thanks!",
      "Bless you!",
      "I appreciate your prompt attention.",
      "Thanks for your consideration.",
      "Thank you for your flexibility!",
      "Really appreciate your input.",
      "A big thanks to you!",
      "Thank you, this is great!",
      "Thanks, you're a star!",
      "I'm very grateful.",
      "Thanks for your suggestion!",
      "Thank you, that's exactly what I needed!",
      "Appreciate your expertise!",
      "Thanks for listening!",
      "Kudos!",
      "Thank you for your thoroughness.",
      "Thank you, that helps a lot!",
      "Thanks, I really needed that!",
      "Thanks for the heads up!",
      "Thanks for your prompt reply!",
      "Truly appreciate your feedback.",
      "Thanks for the clarification!",
      "You're a gem, thanks!",
      "Thank you for your time and effort.",
      "Thanks for taking care of that.",
      "Greatly appreciate your help.",
      "Thank you for your attention.",
      "Thanks for understanding!",
      "I'm indebted to you.",
      "Wonderful job, thanks!",
      "Thanks for your diligence.",
      "Thanks for resolving this quickly.",
      "Thank you, I couldn't have done it without you.",
      "A huge thank you!",
      "Thank you for your promptness.",
      "You've been incredibly helpful, thanks!",
      "Thank you so much for your support!",
      "Your help is greatly valued.",
      "Sincerely, thanks!",
      "Thank you for your assistance.",
      "Thanks for making this easy!",
      "Deeply grateful for your help.",
      "I appreciate your commitment!",
      "Big thumb up!",
      "Thanks for your patience and expertise.",
      "Appreciate your understanding.",
      "Thank you, it means a lot to me.",
      "Thank you for this opportunity.",
      "Thank you for reaching out!",
      "Your support has been invaluable.",
      "Thank you for your clarifica

In [71]:
cl = " ".join([f" - {x}\n" for x in c])

fl = " ".join([f" - {x}\n" for x in f])

ll = " ".join([f" - {x}\n" for x in l])

ncl = " ".join([f" - {x}\n" for x in nc])

hl = " ".join([f" - {x}\n" for x in h])

ccl = " ".join([f" - {x}\n" for x in cc])

gl = " ".join([f" - {x}\n" for x in g])

gl

" - Hi, how's it going?\n  - Hey! 👋\n  - Hi! I'm excited to chat with you.\n  - Hello! What's up?\n  - Hey there! 😊\n  - Hi, it's nice to meet you.\n  - Hey, how are you today?\n  - Hi! I'm looking forward to talking to you.\n  - Hey! What's new with you?\n  - Hi, how's your day going?\n  - Hey! I'm happy to connect with you.\n  - Hi! It's great to meet you.\n  - Hey, what brings you here?\n  - Hi! I'm excited to get to know you.\n  - Hey! Long time no talk!\n  - Hi! I'm looking forward to chatting with you.\n  - Hey! How's life treating you?\n  - Hi! Let's catch up.\n  - Hey! What's on your mind?\n  - Hi! It's nice to reconnect.\n  - Hey! Let's talk soon.\n  - Hi! I'm glad we connected.\n  - Hey there! 😊 How's it going?\n  - Good morning! 🌞 Hope you're well!\n  - Hi! 👋 What's up?\n  - Hello! How are you today? 😄\n  - Yo! What's good? 😎\n  - Hey! Long time no chat! 🕰️\n  - Hi! How's everything? 🌟\n  - Hey, what's new? 🤔\n  - Good evening! 🌙 How was your day?\n  - Hey! What's happening?

In [63]:
c = [
        "What is the company culture like?",
        "I'm interested in the customer service role, can you send me more info?",
        "Do you offer any training programs for new hires?",
        "What are the working hours for the inbound sales team?",
        "How many days off do we get per year? 🤔",
        "Is there opportunity for growth within the company?",
        "What kind of benefits do you offer to employees?",
        "I'm interested in the data entry role, what are the responsibilities?",
        "Do you have a referral program for employees?",
        "What is the average salary for a customer service rep?",
        "Do you have a dress code policy?",
        "How does the performance evaluation process work?",
        "What kind of support does the company offer for professional development?",
        "Can you tell me more about the team I'll be working with?",
        "What are the company's short-term and long-term goals?",
        "Are there any opportunities for remote work?",
        "What's the typical career path for someone in the customer service team?",
        "Do you offer any employee discounts or perks?",
        "How does the company approach work-life balance?",
        "What sets this company apart from other call centers?",
        "What's the company's policy on flexible scheduling?",
        "Hey there! Could you tell me more about the company's history?",
        "Good morning! What are the company's main products or services?",
        "Hi! Can you provide some insights into the company culture?",
        "Hello! How does the company foster employee growth and development?",
        "Yo! What's the company's mission and values?",
        "Hey! I'm curious, what sets this company apart from its competitors?",
        "Hi! What's the company's stance on sustainability?",
        "Hey, what's the work environment like at the company?",
        "Hey! What's the company's approach to diversity and inclusion?",
        "Hi! Can you tell me about the company's financial performance?",
        "Hello! How does the company give back to the community?",
        "Hey, what's the average tenure of employees at the company?",
        "Hi! How does the company ensure a safe and inclusive workplace?",
        "Hey! Can you share any upcoming projects or initiatives the company is working on?",
        "Hello! What's the company's policy on remote work?",
        "Hi! How does the company encourage innovation among its employees?",
        "Hey! Can you tell me about the company's customer base?",
        "Hello! What training and development opportunities does the company provide?",
        "Hey, what benefits does the company offer to its employees?",
        "Hey! Can you provide some information about the company's technology stack?",
        "Hello! How does the company ensure data privacy and security?",
        "Hi! What's the company's policy on flexible working hours?",
        "Hey! I'm interested to know about the company's training programs.",
        "Hi! Can you tell me about the company's customer satisfaction ratings?",
        "Hey, what opportunities for professional development does the company offer?",
        "Hello! How does the company celebrate employee achievements?",
        "Hi! What's the company's approach to employee wellness?",
        "Hey! Can you share any success stories or case studies?",
        "Hello! How does the company support employee mental health?",
        "Hi! What's the company's strategy for growth and expansion?",
        "Can you tell me more about the company's culture?",
        "What's the typical career path for someone in this role?",
        "How do you measure success in this position?",
        "What are the main challenges new hires usually face?",
        "Is there room for growth within the company?",
        "What kind of training programs do you offer?",
        "Can you explain the team structure here?",
        "How do you handle employee feedback?",
        "What's the most rewarding part of working here?",
        "What are the shift patterns like?",
        "Do you offer remote working options?",
        "How do you support work-life balance?",
        "What are the next steps in the hiring process?",
        "Can you describe a typical day in this job?",
        "How does this role contribute to the company's goals?",
        "What's the company's stance on overtime?",
        "Do you provide any mental health support for employees?",
        "How often do you conduct performance reviews?",
        "What's the team dynamic like?",
        "Are there opportunities for advancement?",
        "What's the company's turnover rate?",
        "Can you tell me more about the company’s mission?",
        "How does the company recognize and reward outstanding performance?",
        "What are the biggest challenges the company is currently facing?",
        "How do you handle conflict within the team?",
        "What's the onboarding process like?",
        "How are goals and expectations communicated to the team?",
        "What kind of feedback can I expect from my supervisor?",
        "What's your policy on work-from-home?",
        "How diverse is your team?",
        "Are there opportunities for cross-training or job rotation?",
        "What tools and technologies will I be using?",
        "How does the company support professional development?",
        "What's the company’s approach to customer service?",
        "Do you have a mentoring program?",
        "What are the main goals for the team this year?",
        "Can you tell me about a successful project the team has completed recently?",
        "What type of clients or customers does the company serve?",
        "How does the company handle high-stress periods?",
        "What's the most common reason employees stay long-term?",
        "Do you offer any tuition reimbursement programs?",
        "How would you describe the company’s management style?",
        "What are the key skills needed for success in this role?",
        "What are the company's values?",
        "How do you ensure employees feel valued?",
        "Is there a dress code?",
        "How soon can I expect to hear back after this interview?",
        "What are the benefits and perks?",
        "How do you keep employees motivated and engaged?",
        "Do you celebrate achievements or milestones?"
        
    ]


f = ["Can you please update me on my application status?",
        "What's the current stage of my hiring process?",
        "Feedback on my interview would be appreciated, thanks!",
        "How's my application doing? 🤔",
        "Any news on my job application?",
        "Could you share an update on my candidacy?",
        "What's the latest on my application?",
        "Kindly inform me about my application progress.",
        "Can I get an update on my interview?",
        "I'd love to know where I stand in the process.",
        "Is there any update on my application status?",
        "Please let me know if I'm still in the running?",
        "Can you give me some feedback on my resume?",
        "What's the current status of my application?",
        "Any feedback on my performance during the interview?",
        "Can I get an update on the hiring process?",
        "I'm eagerly waiting to hear about my application.",
        "What's the decision on my application?",
        "Is my application still being considered?",
        "Am I still in the running for this position?",
        "Hey there! Just checking in to see if there's any update on my interview status?",
        "Good morning! I was wondering if there's any news on where I stand in the evaluation process.",
        "Hi! Can you provide me with an update on the status of my application, please?",
        "Hello! I hope you're doing well. Could you please let me know the current status of my interview?",
        "Yo! Any chance I could get an update on how things are progressing with my evaluation?",
        "Hey! Just wanted to touch base and inquire about the status of my application.",
        "Hi! I hope all is going smoothly. Any updates on my interview status?",
        "Hello! Hope you're having a great day. Can you give me an update on my evaluation status?",
        "Hey there! Wondering if there's any news on my application status?",
        "Good morning! I hope you're doing well. Could you provide me with an update on my interview status?",
        "Hi! Just checking in to see if there's any progress with my evaluation.",
        "Hello! Hope you're having a good day. Any updates on my interview status?",
        "Hey! Any chance I could get an update on my application status?",
        "Hi! I hope you're doing well. Can you please let me know the status of my evaluation?",
        "Hello! Just reaching out to see if there's any update on my interview status.",
        "Hey there! Hoping to get an update on my application status.",
        "Hi! I'm eager to know the status of my evaluation. Any updates?",
        "Hello! Can you please provide me with an update on my interview status?",
        "Hey! Just wanted to follow up on my application status. Any news?",
        "What's the update on my application?",
        "Can you tell me where my evaluation stands?",
        "What's the current status of my interview process?",
        "How's my file looking?",
        "Any news on the selection process?",
        "Where are we at with the flow?",
        "What's the holdup?",
        "Can I get an update on the review process?",
        "How much longer til I hear back?",
        "What's the ETA on the next steps?",
        "Can someone give me a status update?",
        "What's the progress report?",
        "Am I close to getting an offer?",
        "What's the deal with my application?",
        "Can I get some clarity on the evaluation process?",
        "How's the process going?",
        "When can I expect to hear back?",
        "What's taking so long?",
        "Can you shed some light on the current status?",
        "What's the most recent update?",
        "How much longer do I have to wait?",
        "Can you walk me through the next steps?",
        "What's the latest on my evaluation?",
        "What's the plan moving forward?",
        "Can someone give me a heads up on the timeline?",
        "What's the status of the review?",
        "Am I still in the running?",
        "What's the hold up?",
        "Can I get a quick update?",
        "What's going on with the process?",
        "How's the team doing on the evaluation?",
        "When can I expect to hear some news?",
        "What's the word on my application?",
        "Can someone give me a status report?",
        "What's the current state of affairs?",
        "How's the flow going?",
        "What's the latest development?",
        "Can you give me a progress report?",
        "What's the deal with the delay?",
        "Am I still being considered?",
        "What's the status of the interview process?"]


l = [ "I'm taking a break, talk to you later on WhatsApp!",
        "I need to go, catch you later!",
        "I'm feeling a bit tired, will continue this convo later 😴",
        "I'm out for now, will get back to you later!",
        "Have to run, talk to you soon!",
        "I'm not feeling up to chat right now, will continue later",
        "I'm stepping away for a bit, be back soon!",
        "I'm bored with this convo, will catch you later",
        "I have to go, but we can pick this up later",
        "I'm not able to continue right now, will talk to you later",
        "I need some time to myself, talk to you later!",
        "I'm taking a time-out, will come back to this later",
        "I'm not feeling the energy to chat, will catch you later",
        "I have to go, but we can continue this on WhatsApp later",
        "I'm disconnecting for now, talk to you soon!",
        "I'm not able to focus, will pick this up later",
        "I'm out of energy, will catch you later!",
        "I need a break from this convo, will continue later",
        "I'm stepping away, but we can continue this on WhatsApp later",
        "I'm not able to chat right now, will talk to you soon!",
        "I'm feeling drained, will continue this later",
        "Hey there! Thanks for reaching out. I'm currently tied up with a meeting, can we continue this later?",
        "Good morning! I'm in the middle of something right now, can we pick this up in an hour?",
        "Hi! Sorry, I'm swamped at the moment. Can we chat about this tomorrow?",
        "Hello! I'm unable to continue the process at the moment due to some unforeseen circumstances.",
        "Yo! Can we put a pin on this for now? I'll get back to you as soon as I can.",
        "Hey! Unfortunately, I won't be able to proceed further with the interview today. Can we reschedule?",
        "Hi! I'm tied up with back-to-back meetings. Can we touch base later?",
        "Hello! I'm currently on the go, so I'll have to get back to you about this later.",
        "Hey there! I'm really sorry, but I have an urgent task to attend to. Can we resume this discussion tomorrow?",
        "Good morning! I'm running a bit behind schedule today. Can we continue this conversation later?",
        "Hi! I'm afraid I won't be able to proceed with the evaluation at the moment. Can we reconvene tomorrow?",
        "Hello! I'm currently dealing with an emergency situation. Can we postpone this to a later time?",
        "Hey! I'm really sorry, but I have to step away for a moment. Can we continue this in an hour?",
        "Hi! Unfortunately, I've hit a roadblock in the process. Can we revisit this tomorrow?",
        "Hey there! I'm in the middle of something important. Can we catch up on this later?",
        "Hello! I'm sorry, but I'm unable to dedicate time to this right now. Can we discuss this tomorrow?",
        "Hi! I'm currently occupied with a pressing matter. Can we pick this up later in the day?",
        "Hey! I'm juggling multiple tasks at the moment. Can we continue this at a more convenient time?",
        "Hi! I'm really sorry, but I have to step out unexpectedly. Can we continue this discussion later?",
        "Hello! I'm tied up with a deadline right now. Can we revisit this tomorrow?",
        "Can we catch up later? 😊",
        "I'm running behind, can we reschedule?",
        "I'm slammed today, can we talk later?",
        "I don't have time right now, sorry!",
        "I'm not available at the moment, can we schedule for later?",
        "Maybe later today? My schedule is crazy!",
        "I'm in the middle of something, can we talk soon?",
        "I'm on a deadline, can we catch up later?",
        "Not a good time for me, can we reschedule?",
        "I'm tied up right now, can we talk later?",
        "I don't have a minute to spare, can we talk soon?",
        "Let's talk later, I'm swamped!",
        "I'm not free at the moment, can we schedule for later?",
        "I'm in a rush, can we catch up soon?",
        "I need some time, can we talk later?",
        "Can we chat later? I'm on a call!",
        "I'm running low on time, can we talk soon?",
        "I'm not available now, can we schedule for later?",
        "Maybe some other time? I'm busy!",
        "I'm on a tight deadline, can we talk later?",
        "I don't have the bandwidth for that right now!",
        "Let's talk later, I'm in a meeting!",
        "I'm in the middle of something, can we catch up soon?",
        "Not a good time for me, can we reschedule?",
        "I'm slammed with work, can we talk later?",
        "I need some time to focus, can we talk soon?",
        "I'm on a roll, can we catch up later?",
        "I'm not free to chat right now, sorry!",
        "Can we talk later? I'm in the middle of something!",
        "I'm running out of time, can we talk soon?",
        "I'm not available at the moment, can we schedule later?",
        "Maybe later today? I'm crazy busy!",
        "I'm tied up with work, can we catch up later?",
        "I don't have time to chat, sorry!",
        "Let's talk later, I'm on a deadline!",
        "I need some time to focus, can we talk later?",
        "Can we catch up soon? I'm slammed!",
        "I'm not free to chat right now, can we talk later?"]


nc = ["Thanks for the offer, but I’ll pass this time.",
        "I appreciate it, but I’m not interested in continuing.",
        "I'm going to have to decline, thanks for considering me.",
        "Sorry, but I don’t agree with this approach.",
        "I’ve decided not to move forward with this process.",
        "Thanks, but I’m not up for it.",
        "Unfortunately, I have to say no.",
        "I’m not comfortable with this, sorry.",
        "I’m out, but thanks for the opportunity.",
        "I’ve thought about it, and I’m not interested.",
        "Thanks, but this isn't for me.",
        "I'm going to step back from this, but thank you.",
        "I’m choosing not to proceed, thanks.",
        "This doesn't align with my interests, so I’m not continuing.",
        "I’ll be bowing out of this process.",
        "I’m going to pass on this, thanks though.",
        "No, I’m not going to accept this.",
        "I can’t go along with this plan.",
        "I have to say no, but I appreciate it.",
        "Not agreeing with this, sorry.",
        "I’m not interested in pursuing this further.",
        "I’m going to have to pass on this opportunity.",
        "I won’t be moving forward with this, thanks.",
        "This isn’t something I want to do, sorry.",
        "I’m not on board with this.",
        "Nope, not for me. Thanks anyway!",
        "I’m not feeling this, so I’m out.",
        "I’m going to decline, but I appreciate the offer.",
        "This doesn’t work for me, so I’m saying no.",
        "I’ve decided against this, thanks anyway.",
        "I’m not continuing with this, sorry.",
        "I’m declining to move forward, thanks.",
        "Thanks, but I’ll have to decline.",
        "This isn't the right fit for me, so no.",
        "I don't agree with this, so I won't proceed.",
        "I’m going to have to say no to this.",
        "Sorry, but I’m not interested in this.",
        "Thanks, but I’m not going ahead with this.",
        "I’m passing on this, but thanks.",
        "I’m choosing not to participate, thank you.",
        "I'm not interested, sorry!",
        "I don't think so!",
        "Not for me, thanks!",
        "I'm out!",
        "I don't agree!",
        "Not my thing!",
        "I'm not comfortable with that!",
        "No way, sorry!",
        "I'm not on board with that!",
        "I'll have to pass!",
        "Not interested!",
        "I don't want to continue!",
        "I'm not down for that!",
        "I'm good, thanks!",
        "No thanks!",
        "I'll have to say no!",
        "That's not for me!",
        "I'm not into that!",
        "I don't want to do that!",
        "I'm not up for it!",
        "Not my style!",
        "I'm not feeling it!",
        "I'll have to decline!",
        "I don't think that's for me!",
        "Nope, not interested!",
        "I'm not sure I want to!",
        "I'm not ready for that!",
        "I'll have to say no, thanks!",
        "That doesn't work for me!",
        "I'm not comfortable with that!",
        "I don't think I can do that!",
        "Nah, I'm good!",
        "I'll have to pass, sorry!",
        "I don't want to go further!",
        "I'm not on the same page!",
        "I don't agree with that!",
        "I'll have to say no way!",
        "I don't think that's a good idea!",
        "No thanks, not for me!",
        "I'll have to decline, thanks!",
        "I'm not up for that, sorry!",
        "That's not something I want to do!",
        "I don't want to proceed!",
        "I'll have to say no, sorry!",
        "I'm not interested, thanks!",]


h = [
    "Got a knack for troubleshooting tech issues.",
      "Always up for a challenge in coding.",
      "Enjoys working in diverse teams.",
      "Aims to climb the career ladder in tech.",
      "Handles customer queries with patience.",
      "Loves learning new programming languages.",
      "Believes in continuous improvement.",
      "Strives to exceed sales targets.",
      "Enjoys solving complex problems.",
      "Values teamwork and collaboration.",
      "Aims to become a team leader.",
      "Enjoys helping customers find solutions.",
      "Passionate about tech innovations.",
      "Believes in delivering quality service.",
      "Aims to improve customer satisfaction.",
      "Enjoys working in a fast-paced environment.",
      "Values feedback and constructive criticism.",
      "Aims to develop leadership skills.",
      "Enjoys learning about new technologies.",
      "Believes in a customer-first approach.",
      "Aims to achieve personal and professional growth.",
      "Enjoys working on challenging projects.",
      "Values diversity and inclusion.",
      "Aims to make a positive impact.",
      "Enjoys working with cutting-edge technology.",
      "Believes in the power of teamwork.",
      "Aims to become an expert in the field.",
      "Enjoys helping others succeed.",
      "Values integrity and honesty.",
      "Aims to build strong customer relationships.",
      "Enjoys working in a collaborative environment.",
      "Believes in the importance of communication.",
      "Aims to stay updated with industry trends.",
      "Enjoys finding innovative solutions.",
      "Values hard work and dedication.",
      "Aims to achieve excellence in all tasks.",
      "Enjoys mentoring new team members.",
      "Believes in the value of continuous learning.",
      "Aims to contribute to the company's success.",
      "Enjoys working on cross-functional teams.",
      "Values transparency and openness.",
      "Aims to develop new skills and knowledge.",
      "Enjoys tackling new challenges.",
      "Believes in the importance of customer feedback.",
      "Aims to deliver exceptional service.",
      "Enjoys working in a dynamic environment.",
      "Values respect and professionalism.",
      "Aims to achieve career growth.",
      "Enjoys learning from experiences.",
      "Believes in the power of positive thinking.",
      "Aims to be a role model for others.",
      "Enjoys working with a diverse team.",
      "Values creativity and innovation.",
      "Aims to exceed customer expectations.",
      "Enjoys taking on new responsibilities.",
      "Believes in the importance of work-life balance.",
      "Aims to be a top performer.",
      "Enjoys working on customer-centric projects.",
      "Values empathy and understanding.",
      "Aims to achieve personal goals.",
      "Enjoys collaborating with others.",
      "Believes in the value of teamwork.",
      "Aims to be a reliable team member.",
      "Enjoys learning from feedback.",
      "Values commitment and perseverance.",
      "Aims to deliver high-quality work.",
      "Enjoys working in a supportive environment.",
      "Believes in the importance of customer satisfaction.",
      "Aims to be a proactive problem solver.",
      "Enjoys working on innovative projects.",
      "Values honesty and integrity.",
      "Aims to achieve professional excellence.",
      "Enjoys helping customers with their needs.",
      "Believes in the power of collaboration.",
      "Aims to be a positive influence.",
      "Enjoys working in a customer-focused role.",
      "Values dedication and hard work.",
      "Aims to achieve success in the industry.",
      "Enjoys learning new skills.",
      "Believes in the importance of adaptability.",
      "Aims to be a valuable team member.",
      "Enjoys working on challenging tasks.",
      "Values teamwork and cooperation.",
      "Aims to deliver outstanding results.",
      "Enjoys helping others achieve their goals.",
      "Believes in the value of continuous improvement.",
      "Aims to be a top contributor.",
      "Enjoys working in a fast-paced setting.",
      "Values respect and collaboration.",
      "Aims to achieve career milestones.",
      "Enjoys learning from challenges.",
      "Believes in the importance of customer service.",
      "Aims to be a dependable team member.",
      "Enjoys working on customer-focused projects.",
      "Values innovation and creativity.",
      "Aims to exceed performance expectations.",
      "Enjoys taking on new challenges.",
      "Believes in the power of teamwork.",
      "Aims to be a positive role model.",
      "Enjoys working in a dynamic team.",
      "Values commitment and dedication.",
      "Aims to achieve professional growth.",
      "Enjoys learning from experiences.",
      "Believes in the importance of customer feedback.",
      "Aims to deliver exceptional service.",
      "Enjoys working in a collaborative environment.",
      "Values transparency and honesty.",
      "Aims to develop new skills.",
      "Enjoys tackling new projects.",
      "Believes in the value of customer satisfaction.",
      "Aims to be a proactive team member.",
      "Enjoys working on innovative solutions.",
      "Values integrity and professionalism.",
      "Aims to achieve excellence in all tasks.",
      "Enjoys mentoring others.",
      "Believes in the power of positive thinking.",
      "Aims to contribute to team success.",
      "Enjoys working with a diverse team.",
      "Values creativity and innovation.",
      "Aims to exceed customer expectations.",
      "Enjoys taking on new responsibilities.",
      "Believes in the importance of work-life balance.",
      "Aims to be a top performer.",
      "Enjoys working on customer-centric projects.",
      "Values empathy and understanding.",
      "Aims to achieve personal goals.",
      "Enjoys collaborating with others.",
      "Believes in the value of teamwork.",
      "Aims to be a reliable team member.",
      "Enjoys learning from feedback.",
      "Values commitment and perseverance.",
      "Aims to deliver high-quality work.",
      "Enjoys working in a supportive environment.",
      "Believes in the importance of customer satisfaction.",
      "Aims to be a proactive problem solver.",
      "Enjoys working on innovative projects.",
      "Values honesty and integrity.",
      "Aims to achieve professional excellence.",
      "Enjoys helping customers with their needs.",
      "Believes in the power of collaboration.",
      "Aims to be a positive influence.",
      "Enjoys working in a customer-focused role.",
      "Values dedication and hard work.",
      "Aims to achieve success in the industry.",
      "Enjoys learning new skills.",
      "Believes in the importance of adaptability.",
      "Aims to be a valuable team member.",
      "Enjoys working on challenging tasks.",
      "Values teamwork and cooperation.",
      "Aims to deliver outstanding results.",
      "Enjoys helping others achieve their goals.",
      "Believes in the value of continuous improvement.",
      "Aims to be a top contributor.",
      "Enjoys working in a fast-paced setting.",
      "Values respect and collaboration.",
      "Aims to achieve career milestones.",
      "Enjoys learning from challenges.",
      "Believes in the importance of customer service.",
      "Aims to be a dependable team member.",
      "Enjoys working on customer-focused projects.",
      "Values innovation and creativity.",
      "Aims to exceed performance expectations.",
      "Enjoys taking on new challenges.",
      "Believes in the power of teamwork.",
      "Aims to be a positive role model.",
      "Enjoys working in a dynamic team.",
      "Values commitment and dedication.",
      "Aims to achieve professional growth.",
      "Enjoys learning from experiences.",
      "Believes in the importance of customer feedback.",
      "Aims to deliver exceptional service.",
      "Enjoys working in a collaborative environment.",
      "Values transparency and honesty.",
      "Aims to develop new skills.",
      "Enjoys tackling new projects.",
      "Believes in the value of customer satisfaction.",
      "Aims to be a proactive team member.",
      "Enjoys working on innovative solutions.",
      "Values integrity and professionalism.",
      "Aims to achieve excellence in all tasks.",
      "Enjoys mentoring others.",
      "Believes in the power of positive thinking.",
      "Aims to contribute to team success.",
      "Enjoys working with a diverse team.",
      "Values creativity and innovation.",
      "Aims to exceed customer expectations.",
      "Enjoys taking on new responsibilities.",
      "Believes in the importance of work-life balance.",
      "Aims to be a top performer.",
      "Enjoys working on customer-centric projects.",
      "Values empathy and understanding.",
      "Aims to achieve personal goals.",
      "Enjoys collaborating with others.",
      "Believes in the value of teamwork.",
      "Aims to be a reliable team member.",
      "Enjoys learning from feedback.",
      "Values commitment and perseverance.",
      "Aims to deliver high-quality work.",
      "Enjoys working in a supportive environment.",
      "Believes in the importance of customer satisfaction.",
      "Aims to be a proactive problem solver.",
      "Enjoys working on innovative projects.",
      "Values honesty and integrity.",
      "Aims to achieve professional excellence.",
      "Enjoys helping customers with their needs.",
      "Believes in the power of collaboration.",
      "Aims to be a positive influence.",
      "Enjoys working in a customer-focused role.",
      "Values dedication and hard work.",
      "Aims to achieve success in the industry.",
      "Enjoys learning new skills.",
      "Believes in the importance of adaptability.",
      "Aims to be a valuable team member.",
      "Enjoys working on challenging tasks.",
      "Values teamwork and cooperation.",
      "Aims to deliver outstanding results.",
      "Enjoys helping others achieve their goals.",
      "Believes in the value of continuous improvement.",
      "Aims to be a top contributor.",
      "Enjoys working in a fast-paced setting.",
      "Values respect and collaboration.",
      "Aims to achieve career milestones.",
      "Enjoys learning from challenges.",
      "Believes in the importance of customer service.",
      "Aims to be a dependable team member.",
      "Enjoys working on customer-focused projects.",
      "Values innovation and creativity.",
      "Aims to exceed performance expectations.",
      "Enjoys taking on new challenges.",
      "Believes in the power of teamwork.",
      "Aims to be a positive role model.",
      "Enjoys working in a dynamic team.",
      "Values commitment and dedication.",
      "Aims to achieve professional growth.",
      "Enjoys learning from experiences.",
      "Believes in the importance of customer feedback.",
      "Aims to deliver exceptional service.",
      "Enjoys working in a collaborative environment.",
      "Values transparency and honesty.",
      "Aims to develop new skills.",
      "Enjoys tackling new projects.",
      "Believes in the value of customer satisfaction.",
      "Aims to be a proactive team member.",
      "Enjoys working on innovative solutions.",
      "Values integrity and professionalism.",
      "Aims to achieve excellence in all tasks.",
      "Enjoys mentoring others.",
      "Believes in the power of positive thinking.",
      "Aims to contribute to team success.",
      "Enjoys working with a diverse team.",
      "Values creativity and innovation.",
      "Aims to exceed customer expectations.",
      "Enjoys taking on new responsibilities.",
      "Believes in the importance of work-life balance.",
      "Aims to be a top performer.",
      "Enjoys working on customer-centric projects.",
      "Values empathy and understanding.",
      "Aims to achieve personal goals.",
      "Enjoys collaborating with others.",
      "Believes in the value of teamwork.",
      "Aims to be a reliable team member.",
      "Enjoys learning from feedback.",
      "Values commitment and perseverance.",
      "Aims to deliver high-quality work.",
      "Enjoys working in a supportive environment.",
      "Believes in the importance of customer satisfaction.",
      "Aims to be a proactive problem solver.",
      "Enjoys working on innovative projects.",
      "Values honesty and integrity.",
      "Aims to achieve professional excellence.",
      "Enjoys helping customers with their needs.",
      "Believes in the power of collaboration.",
      "Aims to be a positive influence.",
      "Enjoys working in a customer-focused role.",
      "Values dedication and hard work.",
      "Aims to achieve success in the industry.",
      "Enjoys learning new skills.",
      "Believes in the importance of adaptability.",
      "Aims to be a valuable team member.",
      "Enjoys working on challenging tasks.",
      "Values teamwork and cooperation.",
      "Aims to deliver outstanding results.",
      "Enjoys helping others achieve their goals.",
      "Believes in the value of continuous improvement.",
      "Aims to be a top contributor.",
      "Enjoys working in a fast-paced setting.",
      "Values respect and collaboration.",
      "Aims to achieve career milestones.",
      "Enjoys learning from challenges.",
      "Believes in the importance of customer service.",
      "Aims to be a dependable team member.",
      "Enjoys working on customer-focused projects.",
      "Values innovation and creativity.",
      "Aims to exceed performance expectations.",
      "Enjoys taking on new challenges.",
      "Believes in the power of teamwork.",
      "Aims to be a positive role model.",
      "Enjoys working in a dynamic team.",
      "Values commitment and dedication.",
      "Aims to achieve professional growth.",
      "Enjoys learning from experiences.",
      "Believes in the importance of customer feedback.",
      "Aims to deliver exceptional service.",
      "Enjoys working in a collaborative environment.",
      "Values transparency and honesty.",
      "Aims to develop new skills.",
      "Enjoys tackling new projects.",
      "Believes in the value of customer satisfaction.",
      "Aims to be a proactive team member.",
      "Enjoys working on innovative solutions.",
      "Values integrity and professionalism.",
      "Aims to achieve excellence in all tasks.",
      "Enjoys mentoring others.",
      "Believes in the power of positive thinking.",
      "Aims to contribute to team success.",
      "Enjoys working with a diverse team.",
      "Values creativity and innovation.",
      "Aims to exceed customer expectations.",
      "Enjoys taking on new responsibilities.",
      "Believes in the importance of work-life balance.",
      "Aims to be a top performer.",
      "Enjoys working on customer-centric projects.",
      "Values empathy and understanding.",
      "Aims to achieve personal goals.",
      "Enjoys collaborating with others.",
      "Believes in the value of teamwork.",
      "Aims to be a reliable team member.",
      "Enjoys learning from feedback.",
      "Values commitment and perseverance.",
      "Aims to deliver high-quality work.",
      "Enjoys working in a supportive environment.",
      "Believes in the importance of customer satisfaction.",
      "Aims to be a proactive problem solver.",
      "Enjoys working on innovative projects.",
      "Values honesty and integrity.",
      "Aims to achieve professional excellence.",
      "Enjoys helping customers with their needs.",
      "Believes in the power of collaboration.",
      "Aims to be a positive influence.",
      "Enjoys working in a customer-focused role.",
      "Values dedication and hard work.",
      "Aims to achieve success in the industry.",
      "Enjoys learning new skills.",
      "Believes in the importance of adaptability.",
      "Aims to be a valuable team member.",
      "Enjoys working on challenging tasks.",
      "Values teamwork and cooperation.",
      "Aims to deliver outstanding results.",
      "Enjoys helping others achieve their goals.",
      "Believes in the value of continuous improvement.",
      "Aims to be a top contributor.",
      "Enjoys working in a fast-paced setting.",
      "Values respect and collaboration.",
      "Aims to achieve career milestones.",
      "Enjoys learning from challenges.",
      "Believes in the importance of customer service.",
      "Aims to be a dependable team member.",
      "Enjoys working on customer-focused projects.",
      "Values innovation and creativity.",
      "Aims to exceed performance expectations.",
      "Enjoys taking on new challenges.",
      "Believes in the power of teamwork.",
      "Aims to be a positive role model.",
      "Enjoys working in a dynamic team.",
      "Values commitment and dedication.",
      "Aims to achieve professional growth.",
      "Enjoys learning from experiences.",
      "Believes in the importance of customer feedback.",
      "Aims to deliver exceptional service.",
      "Enjoys working in a collaborative environment.",
      "Values transparency and honesty.",
      "Aims to develop new skills.",
      "Enjoys tackling new projects.",
      "Believes in the value of customer satisfaction.",
      "Aims to be a proactive team member.",
      "Enjoys working on innovative solutions.",
      "Values integrity and professionalism.",
      "Aims to achieve excellence in all tasks.",
      "Enjoys mentoring others.",
      "Believes in the power of positive thinking.",
      "Aims to contribute to team success.",
      "Enjoys working with a diverse team.",
      "Values creativity and innovation.",
      "Aims to exceed customer expectations.",
      "Enjoys taking on new responsibilities.",
      "Believes in the importance of work-life balance.",
      "Aims to be a top performer.",
      "Enjoys working on customer-centric projects.",
      "Values empathy and understanding.",
      "Aims to achieve personal goals.",
      "Enjoys collaborating with others.",
      "Believes in the value of teamwork.",
      "Aims to be a reliable team member.",
      "Enjoys learning from feedback.",
      "Values commitment and perseverance.",
      "Aims to deliver high-quality work.",
      "Enjoys working in a supportive environment.",
      "Believes in the importance of customer satisfaction.",
      "Aims to be a proactive problem solver.",
      "Enjoys working on innovative projects.",
      "Values honesty and integrity.",
      "Aims to achieve professional excellence.",
      "Enjoys helping customers with their needs.",
      "Believes in the power of collaboration.",
      "Aims to be a positive influence.",
      "Enjoys working in a customer-focused role.",
      "Values dedication and hard work.",
      "Aims to achieve success in the industry.",
      "Enjoys learning new skills.",
      "Believes in the importance of adaptability.",
      "Aims to be a valuable team member.",
      "Enjoys working on challenging tasks.",
      "Values teamwork and cooperation.",
      "Aims to deliver outstanding results.",
      "Enjoys helping others achieve their goals.",
      "Believes in the value of continuous improvement.",
      "Aims to be a top contributor.",
      "Enjoys working in a fast-paced setting.",
      "Values respect and collaboration.",
      "Aims to achieve career milestones.",
      "Enjoys learning from challenges.",
      "Believes in the importance of customer service.",
      "Aims to be a dependable team member.",
      "Enjoys working on customer-focused projects.",
      "Values innovation and creativity.",
      "Aims to exceed performance expectations.",
      "Enjoys taking on new challenges.",
      "Believes in the power of teamwork.",
      "Aims to be a positive role model.",
      "Enjoys working in a dynamic team.",
      "Values commitment and dedication.",
      "Aims to achieve professional growth.",
      "Enjoys learning from experiences.",
      "Believes in the importance of customer feedback.",
      "Aims to deliver exceptional service.",
      "Enjoys working in a collaborative environment.",
]


cc = ["Sounds good! Let's move forward.",
	    "I'm happy to help. Please proceed.",
	    "That's correct! Let's continue 💬",
	    "Got it! Next steps are...",
	    "Affirmative, let's go ahead.",
	    "Excellent, what's the next question?",
	    "That's okay, let's try again.",
	    "I completely agree. What's next?",
	    "That's correct! 👍",
	    "I'm on the same page. Please proceed.",
	    "That's awesome! What's the next step?",
	    "I'm excited to see what's next!",
	    "That sounds like a plan. Let's do it.",
	    "I'm with you on that. What's next?",
	    "That's a great idea! Let's run with it.",
	    "That's a good point. Let's build on that.",
	    "I'm good with that. What's the next move?",
	    "That's the right approach. Let's continue.",
	    "That's a great start! What's next?",
	    "I'm on board with that. Let's proceed.",
	    "That's a fantastic idea! Let's make it happen.",
	    "Oki",
	    "Okay",
	    "Ok",
        "Yes, I would like to continue.",
        "Sure, I'm in!",
        "Absolutely, let's go ahead.",
        "Count me in!",
        "I agree, let's proceed.",
        "I'm on board with this.",
        "Yep, sounds good to me.",
        "Definitely, let's do this.",
        "Okay, I'm ready to move forward.",
        "Alright, let's continue.",
        "Of course, I'm in favor.",
        "👍 I'm in.",
        "Yes, please proceed.",
        "I'm all set to continue.",
        "For sure, let's keep going.",
        "I'm okay with that, let's proceed.",
        "Certainly, I agree.",
        "👌 Let's go for it.",
        "Fine by me, let's move forward.",
        "I'm down for this!",
        "Sounds good, I'm in!",
        "Yes, I agree!",
        "I'm on board!",
        "Works for me!",
        "I accept!",
        "Alright, let's do it!",
        "I'm down!",
        "Sounds like a plan!",
        "Okay, I'm in!",
        "Yeah, I'm good with that!",
        "I'm cool with it!",
        "I agree, let's move forward!",
        "I'm on the same page!",
        "Sounds good to me!",
        "Okay, I accept!",
        "That works for me! 👍",
        "I'm in agreement!",
        "I concur!",
        "I'm down for that!",
        "I'm good with that!",
        "I'm okay with that!",
        "Yeah, that works!",
        "I'm in!",
        "Alright, sounds good!",
        "I'm on board with that!",
        "I'm good, let's proceed!",
        "Sounds legit!",
        "I'm cool with it, thanks!",
        "That sounds good to me!",
        "I accept the offer!",
        "I'm in, thanks!",
        "Yeah, that sounds good!",
        "I'm down with that!",
        "I'm okay with that, thanks!",
        "Alright, I'm in!",
        "Sounds good, I'm good!",
        "I'm good to go!",
        "I'm in agreement, thanks!",
        "I'm down for it!",
        "Yeah, I'm in!",
        "Sounds good, let's do it!",
        "I'm okay with that, yeah!",
        "I'm in, yeah!",
        "That sounds good, I'm in!",
        "I'm good with that, thanks!"
        ]


g = [
     "Hi, how's it going?",
        "Hey! 👋",
        "Hi! I'm excited to chat with you.",
        "Hello! What's up?",
        "Hey there! 😊",
        "Hi, it's nice to meet you.",
        "Hey, how are you today?",
        "Hi! I'm looking forward to talking to you.",
        "Hey! What's new with you?",
        "Hi, how's your day going?",
        "Hey! I'm happy to connect with you.",
        "Hi! It's great to meet you.",
        "Hey, what brings you here?",
        "Hi! I'm excited to get to know you.",
        "Hey! Long time no talk!",
        "Hi! I'm looking forward to chatting with you.",
        "Hey! How's life treating you?",
        "Hi! Let's catch up.",
        "Hey! What's on your mind?",
        "Hi! It's nice to reconnect.",
        "Hey! Let's talk soon.",
        "Hi! I'm glad we connected.",
        "Hey there! 😊 How's it going?",
        "Good morning! 🌞 Hope you're well!",
        "Hi! 👋 What's up?",
        "Hello! How are you today? 😄",
        "Yo! What's good? 😎",
        "Hey! Long time no chat! 🕰️",
        "Hi! How's everything? 🌟",
        "Hey, what's new? 🤔",
        "Good evening! 🌙 How was your day?",
        "Hey! What's happening? 🤷‍♂️",
        "Hello! How have you been? 😊",
        "Hey, what's cracking? 😄",
        "Hey! How you doin'? 😉",
        "Hello! Trust you're doing well. 🙏",
        "Hi! What's going on? 🤔",
        "Hey, how's your day been? 😊",
        "Greetings! 🎉 How's it going?",
        "Hey! Any news? 📰",
        "Hi there! How's your week been? 📅",
        "Hello! Everything good? 👍",
        "Hey! What's the latest? 📢",
        "Hi! How's your morning? ☕",
        "Hey! What are you up to? 🤔",
        "Hi! How's your day going? 😊",
        "Hello! How are things? 🌟",
        "Hey! How's your night? 🌙",
        "Hi! What's up with you? 🤔",
        "Hey! How's life treating you? 🍀",
        "Hello! How's your week going? 📅",
        "Hi! How's everything on your end? 🌟",
        "Hey! What's good with you? 😎",
        "Hi! How have you been feeling? 😊",
        "Hey! How's it going?",
        "Hi! What's up?",
        "Hey, what's new with you?",
        "Hi, how are you?",
        "Yo! Long time no talk!",
        "Hey! What's poppin?",
        "Hi, how's life treating you?",
        "Hey, how's everything?",
        "Sup?",
        "Hey! What's good?",
        "Hi, what's new?",
        "Hey, how's it going?",
        "Hi! Doing well?",
        "Hey! All good?",
        "Hi, what's poppin?",
        "Yo, how's life?",
        "Hey! What's shakin?",
        "Hi, what's new with you?",
        "Hey, not much?",
        "Hi, how's your day?",
        "Hey! What's up, my dude?",
        "Hi, what's good?",
        "Hey! Not much, you?",
        "Hi, all good?",
        "Yo, how's everything?",
        "Hey! What's new?",
        "Hi, doing alright?",
        "Hey! Life's good?",
        "Hi, not much new?",
        "Yo, how's it going?",
        "Hey! What's the haps?",
        "Hi, what's poppin now?",
        "Hey! All good, you?",
        "Hi, nothing much?",
        "Hey! What's shakin, my friend?",
        "Hi, life's good?",
        "Hey! How's life treating you?",
        "Hi, what's the word?",
        "Yo, how's everything going?",
        "Hey! Not much, just chillin?",
        "Hi, what's the haps?",
        "Hey! All cool?",
        "Hi, doing good?",
        "Yo, what's good?",
        "Hey! Life's all good?",
        "Hi, what's up now?",
        "Hey! What's new today?",
        "Hi, all is well?",
        "Yo, what's poppin now? 💬",
        "Hey there!",
      "Good morning, everyone!",
      "What's up?",
      "Hello! How's it going?",
      "Hi folks, hope all's well.",
      "Greetings, friends!",
      "Hey, how are you today?",
      "Hi there, nice to see you!",
      "Hello, everyone! What's new?",
      "Good evening, hope you had a great day.",
      "Hi all! How's everyone doing?",
      "Hey, hope you're all well!",
      "What's good, folks?",
      "Hello all! Any news?",
      "Good afternoon, everyone!",
      "Howdy, partners!",
      "Hi, what are you up to?",
      "Hello! Anything exciting happening?",
      "Good morning! How's the weather?",
      "Hey everyone, what’s happening?",
      "Hey there! How have things been?",
      "Hello! How's everyone this fine day?",
      "Hi friends, any plans for today?",
      "Greetings! Anything special today?",
      "Hello, what's on your mind today?",
      "Hey! How’s everyone managing?",
      "Good day, everyone!",
      "Hi, anything interesting today?",
      "Howdy! How’s it going folks?",
      "Hey there! Looking forward to today?",
      "Hello! How’s the week been treating you?",
      "Hi all! Ready for a new day?",
      "What's up, chatters?",
      "Hello! What are you all doing today?",
      "Howdy! What’s everyone up to?",
      "Good morning! Busy day ahead?",
      "Hey folks! What’s new and exciting?",
      "Hello everyone! How are things?",
      "Hi! What’s everyone talking about today?",
      "Hey, what’s new with everyone?",
      "Hello, everyone! How's life treating you?",
      "Good evening! How was your day?",
      "Hi all! Any interesting news?",
      "Hello! What are we discussing today?",
      "Hey there! Looking forward to the weekend?",
      "Hi everyone! Any gossip?",
      "Hello all! Anything noteworthy?",
      "Hey! How’s your day been so far?",
      "Good day, folks! What's happening?",
      "Hi, busy day ahead?",
      "What’s up, everybody?",
      "Good morning! Coffee, anyone?",
      "Hello there! Plans for today?",
      "Hey! How are folks doing?",
      "Hello, everyone! What’s everyone up to?",
      "Hi all! How's the mood today?",
      "What's new, team?",
      "Howdy, folks! Everything smooth?",
      "Good evening! Fun plans tonight?",
      "Hey all! How’s it hanging?",
      "Hey, what’s the buzz?",
      "Hi everyone! What's the latest?",
      "Hello! Anyone up for some chat?",
      "Hey folks! Anything cool happening?",
      "Hi there! How's everything going?",
      "Good day to you all!",
      "What's everyone doing today?",
      "Hello! Any big plans this week?",
      "Hi, gang! What’s happening?",
      "Hey there! What’s the good word?",
      "What’s up, team?",
      "Hey! How’s everyone’s week so far?",
      "Good evening folks! What’s everyone doing?",
      "Hi everyone! Hope all is well.",
      "Hello, how's everyone doing today?",
      "Hey all! What’s up?",
      "Hi there! How are things shaping up?",
      "Hello! Got any stories to share?",
      "Good morning folks! What’s new?",
      "Hi, everyone! Ready for the day ahead?",
      "Hey there! Anything interesting today?",
      "Hello! What’s everyone thinking about?",
      "Hey everyone! Ready to chat?",
      "Good afternoon folks! Cool news?",
      "Hi all! How’s your week been?",
      "What’s new, group?",
      "Hey folks! Anything exciting going on?",
      "Hi there! Any updates today?",
      "Good evening! Anything fun happening?",
      "Hey! What’s happening today?",
      "Hi everyone! Up for a chat?",
      "Hello! How are you all feeling today?",
      "Hey, how’s it going with everyone?",
      "Good morning! Everyone doing okay?",
      "Howdy, everyone! Share your thoughts?",
      "Hey all! How's your day planning out?",
      "What's good, everyone?",
      "Hi there! Any good news?",
      "Hello! What's your day like?",
      "Hey! Ready to kick off the day?",
      "Good day everyone! What's new?",
      "Hey! What's the topic today?",
      "Hi! Anything cool happening with you?",
      "Hi there! Got anything on your mind?",
      "Hello everyone! What’s going on?",
      "Hey folks! What’s everyone up to today?",
      "Hi all! Catch anything interesting?",
      "Hi friends! What’s on the agenda?",
      "Good evening, how's everybody?",
      "Hello everyone! How's your day?",
      "Hi all! What's the buzz today?",
      "Hey, what're everyone’s thoughts?",
      "Good morning, team! Ready for action?",
      "What’s the latest, folks?",
      "Hey there! Plans for the weekend?",
      "Good afternoon, how’s everyone doing?",
      "Hi all! How are things?",
      "Hello! How is everyone this fine day?",
      "Hey all! What’s cooking today?",
      "Hi, what's everyone talking about?",
      "Hey folks, what's happening?",
      "Good evening! How are things going?",
      "Hey everyone! Got any news?",
      "What's up everyone?",
      "Good day! How are you all?",
      "Hi folks, anything interesting?",
      "Hello! What’s your plan today?",
      "Hey, how are things?",
      "Hi all! How’s life treating you?",
      "Hi there! Any updates?",
      "Hello! Any plans for today?",
      "Hey folks! Up for some chat?",
      "Good morning! Have a fantastic day!",
      "Hi everyone! Ready to start the day?",
      "Hey all! What’s everyone up to?",
      "Hello everyone! What’s the latest gossip?",
      "Howdy! What's everyone up to?",
      "Good day! What’s new, folks?",
      "Hey! How’s everyone doing today?",
      "Hi there! Anything exciting going on?",
      "What's up, everyone?",
      "Good evening all! How are we doing?",
      "Hi folks! Any news?",
      "Hello! Ready for some chat?",
      "Hey everyone! Anything new?",
      "Howdy there, partners!",
      "Hi all! How are things going?",
      "Good morning! How's everyone?",
      "Hey, what’s everyone discussing?",
      "Hi! Everyone ready for a new day?",
      "Hello all! How’s the week been?",
      "Hey folks! Anything fun planned?",
      "Hi there! What’s on your mind?",
      "Hello! Any exciting stories?",
      "Hey all! Anything cool happening?",
      "What's new, folks?",
      "Good day! Ready for a chat?",
      "Hi everyone! Hope you’re all well.",
      "Hey there! What's everyone doing?",
      "Hello everyone! Ready to talk?",
      "Howdy, folks! Any updates?",
      "Good evening! What’s everyone up to?",
      "Hi folks! How’s it going?",
      "Hello! Any plans?",
      "Hey everyone! How’s the weather?",
      "Hi all! What's everyone up to?",
      "Hey there! Have anything interesting?",
      "Hi everyone! Anything on your mind?",
      "Hello! How's everyone?",
      "Hey, folks! How's it going?",
      "Good day! How are things?",
      "Hi all! Anything new happening?",
      "Hello! What’s everyone’s plan?",
      "Hey everyone! Any updates to share?",
      "Hi! Ready for a chat?",
      "What's up, everyone?",
      "Good evening! How’s everyone feeling?",
      "Hi folks! Ready for a good chat?",
      "Hello all! How’s everyone doing?",
      "Hey! What’s going on today?",
      "Good morning, friends!",
      "Hi there! How's everyone?",
      "Hello! What’s new with everyone?",
      "Hey all! Anything interesting today?",
      "Hi everyone! How's the mood?",
      "Hello everyone! Anything happening?",
      "Howdy! How's it going?",
      "Good day! Any plans, folks?",
      "Hey folks! How’s the day?",
      "Hi there! Busy day ahead?",
      "What's everyone talking about?",
      "Hello all! Up for some chat?",
      "Hey there! How’s everyone's day?",
      "Hi everyone! What’s happening?",
      "Hello! Any exciting updates?",
      "Hey all! What’s on your mind?",
      "Good morning! Enjoying the day?",
      "Hi, folks! What’s up?",
      "Hello everyone! What’re your plans?",
      "Hey there! Any news today?",
      "Good day! How’s everyone?",
      "Hi all! Anything to share?",
      "Hello! What’s new?",
      "Hey! How's everyone feeling?",
      "Good evening! Any news?",


      "Hey! 👋",
      "Good morning!",
      "Hi there! 😊",
      "What's up?",
      "Hello!",
      "Hey, how's it going?",
      "Greetings of the day.",
      "Hi! How are you doing?",
      "Hey! Long time no see.",
      "Hello! How's everything?",
      "Hey, are you there?",
      "Hi! What’s new with you?",
      "Good afternoon!",
      "Hey! What’s popping?",
      "What’s up? All good?",
      "Hey! How have you been?",
      "Hi, how is it going?",
      "Good day to you!",
      "Hello! How’s life treating you?",
      "Hi! Any plans for today?",
      "Hey there! Long time, huh?",
      "Good evening!",
      "Hi! What’s up?",
      "Hey! How’s your day going?",
      "Hello there!",
      "How's everything going today?",
      "Hi, what’s new?",
      "Hey! What's happening?",
      "Hello! How are things?",
      "Hey! Ready to chat?",
      "Hi! What's cracking?",
      "Hey! How's your week been?",
      "What’s the latest?",
      "Hi! Everything good?",
      "Hey! It’s been a while.",
      "Hello! How’s it going on your end?",
      "Hey! Anything exciting happening?",
      "Hi! What’s been good?",
      "Hey, got a minute?",
      "Hello! Any news from your side?",
      "Hi! How's your day shaping up?",
      "Hey! What are you up to?",
      "Hello! What’s been happening?",
      "Hi! Got any updates?",
      "Hey! How’s everything going?",
      "What’s shaking?",
      "Hello! What’s new?",
      "Hey! Good to see you online.",
      "Hi! Finally caught you!",
      "Hello! How was your day?",
      "Hey, got some time to chat?",
      "Hi! What's the buzz?",
      "Hey! How's life?",
      "Hello! All good on your side?",
      "Hey, how’s things?",
      "Hi! What’s new and exciting?",
      "Hey! Catch me up!",
      "Hello! How's your day rolling?",
      "Hi! Break time?",
      "Hey! Fill me in!",
      "What’s happening?",
      "Hello! Got any good stories?",
      "Hey! How’s your morning?",
      "Hi! What's your plan today?",
      "Hello! All’s well?",
      "Hey! Share a quick update?",
      "Hi! Been busy?",
      "Hey there! How's it hanging?",
      "Good day!",
      "Hi! Tell me something good!",
      "Hey! How have things been?",
      "Hello! What’s new with you?",
      "Hey! How’s everything?",
      "Hi! Hope you’re well!",
      "Hey! Give me the lowdown.",
      "Hello! What's the latest?",
      "Hi! Any updates from you?",
      "Hey! Anything new happening?",
      "Hi! How was your weekend?",
      "Hey! How’s everything going?",
      "Hello! How’ve you been?",
      "Hey! What's been going on?",
      "Hi! What’s your news?",
      "Hello! Any big plans today?",
      "Hey! Got any gossip?",
      "Hi! Hope it's a good day.",
      "Hey! What's up?",
      "Hello! What's new in your life?",
      "Hi! How’s your afternoon?",
      "Hey! And what’s been up?",
      "Hello! Tell me what’s new.",
      "Hi! Got any plans?",
      "Hey! Ready to chat a bit?",
      "Hello! What’s the good word?",
      "Hi! What’s the vibe today?",
      "Hey! What’s the haps?",
      "Hello! How are things?",
      "Hi! Fill me in!",
      "Hey! Ready to catch up?",
      "Hello! What's been good?",
      "Hi! Anything interesting today?",
      "Hey! How's the weather there?",
      "Hello! How’s your day unfolding?",
      "Hi! What's your status?",
      "Hey! What’s happening?",
      "Hello! Up for some news?",
      "Hi! What’s occurrin'?",
      "Hey! How’s your morning going?",
      "Hello! What’s the latest?",
      "Hi! Any exciting news?",
      "Hey! What have you been up to?",
      "Hello! Any update?",
      "Hi! Everything moving smoothly?",
      "Hey! Long time, no chat!",
      "Hello! What's new and exciting?",
      "Hi! What's good?",
      "Hey! Got some news?",
      "Hello! How you feeling today?",
      "Hi! How’s your vibe?",
      "Hey! What have you been doing?",
      "Hello! What’s shaking?",
      "Hi! How’s it been going?",
      "Hey! Had any fun today?",
      "Hello! Everything alright?",
      "Hi! What’s new in your world?",
      "Hey! How’s it going today?",
      "Hello! Got any news to share?",
      "Hi! What's cooking?",
      "Hey! Give me a heads up.",
      "Hello! What’s up with you?",
      "Hi! How are you keeping?",
      "Hey! How was your night?",
      "Hello! What's the scoop?",
      "Hi! Any big plans this week?",
      "Hey! How’s everything with you?",
      "Hello! How's your weekend?",
      "Hi! What are you up to today?",
      "Hey! How have you been?",
      "Hello! Any buzz?",
      "Hi! What’s happening with you?",
      "Hey! What’s your latest adventure?",
      "Hello! How’s everything with you?",
      "Hi! What’s on your mind?",
      "Hey! Did you hear anything new?",
      "Hello! How’s your evening?",
      "Hi! Got anything to share?",
      "Hey! What’s happening today?",
      "Hello! How’s this week treating you?",
      "Hi! How’s your end?",
      "Hey! What’s on your plate?",
      "Hello! What are you doing?",
      "Hi! What's been new?",
      "Hey! How are you feeling today?",
      "Hello! Up to anything fun?",
      "Hi! What’s the latest with you?",
      "Hey! How are things coming along?",
      "Hello! Got any good news?",
      "Hi! Anything exciting going on?",
      "Hey! How’s your world?",
      "Hello! How’s it coming along?",
      "Hi! What’s your latest update?",
      "Hey! Got time to chat?",
      "Hello! How's your journey?",
      "Hi! Heard anything new?",
      "Hey! Got a story for me?",
      "Hello! What’s happening in your life?",
      "Hi! Got anything fun happening?",
      "Hey! How’s the weather treating you?",
      "Hello! How has your week been?",
      "Hi! Any news lately?",
      "Hey! How's everything over there?",
      "Hello! Anything new transpired?",
      "Hi! Up to anything fun today?",
      "Hey! Got any cool updates?",
      "Hello! How's your new project?",
      "Hi! How’s it rolling?",
      "Hey! Had a good day?",
      "Hello! How's your morning been?",
      "Hi! What's new and interesting?",
      "Hey! What's been happening?",
      "Hello! Any weekend plans?",
      "Hi! What’s shaking?",
      "Hey! What’s the word?",
      "Hello! What’s good with you?",
      "Hi! Got any new plans?",
      "Hey! How’s your life?",
      "Hello! Anything fun going on?",
      "Hi! What's on your docket today?",
      "Hey! How is everything?",
      "Hello! What's the good news?",
      "Hi! Got time to catch up?",
      "Hey! How was your sleep?",
      "Hello! Got any newsworthy updates?",
      "Hi! What’s the plan for today?",
      "Hey! How's your workload?",
      "Hello! How's everything panning out?",
      "Hi! What's up in your world?",
      "Hey! Anything worth mentioning?",
      "Hello! Got time to talk?",
      "Hi! How's your project?",
      "Hey! What's on your agenda?",
      "Hello! What's your day like?",
      "Hi! Got any weekend plans?",
      "Hey! What's on your mind today?",
      "Hello! Got time to chat?",
      "Hi! How’s your schedule today?",
      "Hey! How's the day treating you?",
      "Hello! What’s up?",
      "Hi! Your day going well?",
      "Hey! Got any tales to tell?",
      "Hello! How’s your life?",
      "Hi! What's your schedule like today?",
      "Hey! Any interesting updates?",
      "Hello! What's been happening recently?",
      "Hi! Any good news?",
      "Hey! How's your evening?",
      "Hello! Got something new?",
      "Hi! How's it going for you?",
      "Hey! How's work?",
      "Hello! How’s your afternoon shaping up?",
      "Hi! What's been going on lately?",
      "Hey! How’s your week?",
      "Hello! What are you up to these days?",
      "Hi! Got any interesting news?",
      "Hey! Any fun weekend plans?",
      "Hello! What’s happening with you today?",
      "Hi! What did you do today?",
      "Hey! How’s your workload?",
      "Hello! Any new updates?",
      "Hi! How's your morning so far?",
      "Hey! Got any cool plans?",
      "Hello! Got any big news?",
      "Hi! How’s life treating you today?",
      "Hey! What's your plan for the day?",
      "Hello! How’s it been?",
      "Hi! Any exciting updates?",
      "Hey! What’s new this week?",
      "Hello! What have you been up to?",
      "Hi! Got any weekend plans?",
      "Hey! Give me the deets!",
      "Hello! What’s happening today?",
      "Hi! Any fun updates from your side?",
      "Hey! What’s your latest?",
      "Hello! Got time for a quick chat?",
      "Hi! Got anything new?",
      "Hey! What’s on your plate today?",
      "Hello! What’s occurring?",
      "Hi! Up for a chat?",
      "Hey! How's the vibe?",
      "Hello! How's your week been so far?",
      "Hi! What's in the news?",
      "Hey! Anything good happening?",
      "Hello! How’s your day coming along?",
      "Hi! Got any events today?",
      "Hey! Heard anything interesting?",
      "Hello! How’s your Monday?",
      "Hi! What's the latest news?",
      "Hey! Anything fun planned?",
      "Hello! How’s everything shaping up?",
      "Hi! Got a moment to share?",
      "Hey! How’s your new gig?",
      "Hello! Tell me the news.",
      "Hi! Any weekend adventures?",
      "Hey! What have you been doing lately?",
      "Hello! Got time to chat?",
      "Hi! Got any new projects?",
      "Hey! Got any fun updates?",
      "Hello! What’s the highlight of your day?",
      "Hi! How’s your afternoon?",
      "Hey! What’s the scoop?",
      "Hello! How’s your workload today?",
      "Hi! What's your next move?",
      "Hey! Got anything interesting to share?",
      "Hello! How's your weekend shaping up?",
      "Hi! Got an open moment?",
      "Hey! How’s your new role?",
      "Hello! How’s your day looking?",
      "Hi! What have you been working on?",
      "Hey! How's life on your end?",
      "Hello! What's new in your world?",
      "Hi! Got a minute for updates?",
      "Hey! Any good stories?",
      "Hello! What’s new in your life?",
      "Hi! How has your day been?",
]

In [10]:
import dspy
from typing import List
from datasets import Dataset, DatasetDict
from dotenv import dotenv_values
from rich import print
import random 


greetings = {
  "hello_messages": [
        "Hi, how's it going?",
        "Hey! 👋",
        "Hi! I'm excited to chat with you.",
        "Hello! What's up?",
        "Hey there! 😊",
        "Hi, it's nice to meet you.",
        "Hey, how are you today?",
        "Hi! I'm looking forward to talking to you.",
        "Hey! What's new with you?",
        "Hi, how's your day going?",
        "Hey! I'm happy to connect with you.",
        "Hi! It's great to meet you.",
        "Hey, what brings you here?",
        "Hi! I'm excited to get to know you.",
        "Hey! Long time no talk!",
        "Hi! I'm looking forward to chatting with you.",
        "Hey! How's life treating you?",
        "Hi! Let's catch up.",
        "Hey! What's on your mind?",
        "Hi! It's nice to reconnect.",
        "Hey! Let's talk soon.",
        "Hi! I'm glad we connected.",
        "Hey there! 😊 How's it going?",
        "Good morning! 🌞 Hope you're well!",
        "Hi! 👋 What's up?",
        "Hello! How are you today? 😄",
        "Yo! What's good? 😎",
        "Hey! Long time no chat! 🕰️",
        "Hi! How's everything? 🌟",
        "Hey, what's new? 🤔",
        "Good evening! 🌙 How was your day?",
        "Hey! What's happening? 🤷‍♂️",
        "Hello! How have you been? 😊",
        "Hey, what's cracking? 😄",
        "Hey! How you doin'? 😉",
        "Hello! Trust you're doing well. 🙏",
        "Hi! What's going on? 🤔",
        "Hey, how's your day been? 😊",
        "Greetings! 🎉 How's it going?",
        "Hey! Any news? 📰",
        "Hi there! How's your week been? 📅",
        "Hello! Everything good? 👍",
        "Hey! What's the latest? 📢",
        "Hi! How's your morning? ☕",
        "Hey! What are you up to? 🤔",
        "Hi! How's your day going? 😊",
        "Hello! How are things? 🌟",
        "Hey! How's your night? 🌙",
        "Hi! What's up with you? 🤔",
        "Hello! How's your week going? 📅",
        "Hi! How's everything on your end? 🌟",
        "Hey! What's good with you? 😎",
        "Hi! How have you been feeling? 😊",
        "Hey! How's it going?",
        "Hi! What's up?",
        "Hey, what's new with you?",
        "Hi, how are you?",
        "Yo! Long time no talk!",
        "Hey! What's poppin?",
        "Hi, how's life treating you?",
        "Hey, how's everything?",
        "Sup?",
        "Hey! What's good?",
        "Hi, what's new?",
        "Hey, how's it going?",
        "Hi! Doing well?",
        "Hi, not much new?",
        "Yo, how's it going?",
        "Hey! What's the haps?",
        "Hi, what's poppin now?",
        "Hey! All good, you?",
        "Hi, nothing much?",
        "Hey! What's shakin, my friend?",
        "Hi, life's good?",
        "Yo, how's everything going?",
        "Hey! Not much, just chillin?",
        "Hi, what's the haps?",
        "Hey! All cool?",
        "Hi, doing good?",
        "Yo, what's good?",
        "Hey! Life's all good?",
        "Hi, what's up now?",
        "Hey! What's new today?",
        "Hi, all is well?",
        "Yo, what's poppin now? 💬",
        "Hey there!",
      "Good morning, everyone!",
      "Hello! How's it going?",
      "Hi folks, hope all's well.",
      "Greetings, friends!",
      "Hey, how are you today?",
      "Hi there, nice to see you!",
      "Hello, everyone! What's new?",
      "Good evening, hope you had a great day.",
      "Hi all! How's everyone doing?",
      "Hey, hope you're all well!",
      "What's good, folks?",
      "Hello all! Any news?",
      "Good afternoon, everyone!",
      "Howdy, partners!",
      "Hi, what are you up to?",
      "Hello! Anything exciting happening?",
      "Good morning! How's the weather?",
      "Hey everyone, what’s happening?",
      "Hey there! How have things been?",
      "Hello! How's everyone this fine day?",
      "Hi friends, any plans for today?",
      "Greetings! Anything special today?",
      "Hello, what's on your mind today?",
      "Hey! How’s everyone managing?",
      "Good day, everyone!",
      "Hi, anything interesting today?",
      "Howdy! How’s it going folks?",
      "Hey there! Looking forward to today?",
      "Hello! How’s the week been treating you?",
      "Hi all! Ready for a new day?",
      "What's up, chatters?",
      "Hello! What are you all doing today?",
      "Howdy! What’s everyone up to?",
      "Good morning! Busy day ahead?",
      "Hey folks! What’s new and exciting?",
      "Hello everyone! How are things?",
      "Hi! What’s everyone talking about today?",
      "Hey, what’s new with everyone?",
      "Hello, everyone! How's life treating you?",
      "Good evening! How was your day?",
      "Hi all! Any interesting news?",
      "Hello! What are we discussing today?",
      "Hey there! Looking forward to the weekend?",
      "Hi everyone! Any gossip?",
      "Hello all! Anything noteworthy?",
      "Hey! How’s your day been so far?",
      "Good day, folks! What's happening?",
      "Hi, busy day ahead?",
      "What’s up, everybody?",
      "Good morning! Coffee, anyone?",
      "Hello there! Plans for today?",
      "Hey! How are folks doing?",
      "Hello, everyone! What’s everyone up to?",
      "Hi all! How's the mood today?",
      "What's new, team?",
      "Howdy, folks! Everything smooth?",
      "Good evening! Fun plans tonight?",
      "Hey all! How’s it hanging?",
      "Hey, what’s the buzz?",
      "Hi everyone! What's the latest?",
      "Hello! Anyone up for some chat?",
      "Hey folks! Anything cool happening?",
      "Hi there! How's everything going?",
      "Good day to you all!",
      "What's everyone doing today?",
      "Hello! Any big plans this week?",
      "Hi, gang! What’s happening?",
      "Hey there! What’s the good word?",
      "What’s up, team?",
      "Hey! How’s everyone’s week so far?",
      "Good evening folks! What’s everyone doing?",
      "Hi everyone! Hope all is well.",
      "Hello, how's everyone doing today?",
      "Hey all! What’s up?",
      "Hi there! How are things shaping up?",
      "Hello! Got any stories to share?",
      "Good morning folks! What’s new?",
      "Hi, everyone! Ready for the day ahead?",
      "Hey there! Anything interesting today?",
      "Hello! What’s everyone thinking about?",
      "Hey everyone! Ready to chat?",
      "Good afternoon folks! Cool news?",
      "Hi all! How’s your week been?",
      "What’s new, group?",
      "Hey folks! Anything exciting going on?",
      "Hi there! Any updates today?",
      "Good evening! Anything fun happening?",
      "Hey! What’s happening today?",
      "Hi everyone! Up for a chat?",
      "Hello! How are you all feeling today?",
      "Hey, how’s it going with everyone?",
      "Good morning! Everyone doing okay?",
      "Howdy, everyone! Share your thoughts?",
      "Hey all! How's your day planning out?",
      "What's good, everyone?",
      "Hi there! Any good news?",
      "Hello! What's your day like?",
      "Hey! Ready to kick off the day?",
      "Good day everyone! What's new?",
      "Hey! What's the topic today?",
      "Hi! Anything cool happening with you?",
      "Hi there! Got anything on your mind?",
      "Hello everyone! What’s going on?",
      "Hey folks! What’s everyone up to today?",
      "Hi all! Catch anything interesting?",
      "Hi friends! What’s on the agenda?",
      "Good evening, how's everybody?",
      "Hello everyone! How's your day?",
      "Hi all! What's the buzz today?",
      "Hey, what're everyone’s thoughts?",
      "Good morning, team! Ready for action?",
      "What’s the latest, folks?",
      "Hey there! Plans for the weekend?",
      "Good afternoon, how’s everyone doing?",
      "Hi all! How are things?",
      "Hello! How is everyone this fine day?",
      "Hey all! What’s cooking today?",
      "Hi, what's everyone talking about?",
      "Hey folks, what's happening?",
      "Good evening! How are things going?",
      "Hey everyone! Got any news?",
      "What's up everyone?",
      "Good day! How are you all?",
      "Hi folks, anything interesting?",
      "Hello! What’s your plan today?",
      "Hey, how are things?",
      "Hi all! How’s life treating you?",
      "Hi there! Any updates?",
      "Hello! Any plans for today?",
      "Hey folks! Up for some chat?",
      "Good morning! Have a fantastic day!",
      "Hi everyone! Ready to start the day?",
      "Hey all! What’s everyone up to?",
      "Hello everyone! What’s the latest gossip?",
      "Howdy! What's everyone up to?",
      "Good day! What’s new, folks?",
      "Hey! How’s everyone doing today?",
      "Hi there! Anything exciting going on?",
      "Good evening all! How are we doing?",
      "Hi folks! Any news?",
      "Hello! Ready for some chat?",
      "Hey everyone! Anything new?",
      "Howdy there, partners!",
      "Hi all! How are things going?",
      "Good morning! How's everyone?",
      "Hey, what’s everyone discussing?",
      "Hi! Everyone ready for a new day?",
      "Hello all! How’s the week been?",
      "Hey folks! Anything fun planned?",
      "Hi there! What’s on your mind?",
      "Hello! Any exciting stories?",
      "Hey all! Anything cool happening?",
      "What's new, folks?",
      "Good day! Ready for a chat?",
      "Hi everyone! Hope you’re all well.",
      "Hey there! What's everyone doing?",
      "Hello everyone! Ready to talk?",
      "Howdy, folks! Any updates?",
      "Good evening! What’s everyone up to?",
      "Hi folks! How’s it going?",
      "Hello! Any plans?",
      "Hey everyone! How’s the weather?",
      "Hi all! What's everyone up to?",
      "Hey there! Have anything interesting?",
      "Hi everyone! Anything on your mind?",
      "Hello! How's everyone?",
      "Hey, folks! How's it going?",
      "Good day! How are things?",
      "Hi all! Anything new happening?",
      "Hello! What’s everyone’s plan?",
      "Hey everyone! Any updates to share?",
      "Hi! Ready for a chat?",
      "What's up, everyone?",
      "Good evening! How’s everyone feeling?",
      "Hi folks! Ready for a good chat?",
      "Hello all! How’s everyone doing?",
      "Hey! What’s going on today?",
      "What's everyone talking about?",
      "Hello all! Up for some chat?",
      "Hey there! How’s everyone's day?",
      "Hi everyone! What’s happening?",
      "Hello! Any exciting updates?",
      "Hey all! What’s on your mind?",
      "Good morning! Enjoying the day?",
      "Hi, folks! What’s up?",
      "Hello everyone! What’re your plans?",
      "Hey there! Any news today?",
      "Good day! How’s everyone?",
      "Hi all! Anything to share?",
      "Hello! What’s new?",
      "Hey! How's everyone feeling?",
      "Good evening! Any news?",
      "Hey, how's it going?",
      "Greetings of the day.",
      "Hi! How are you doing?",
      "Hey! Long time no see.",
      "Hello! How's everything?",
      "Hey, are you there?",
      "Hi! What’s new with you?",
      "Good afternoon!",
      "Hi, how is it going?",
      "Good day to you!",
      "Hello! How’s life treating you?",
      "Hi! Any plans for today?",
      "Hey there! Long time, huh?",
      "Good evening!",
      "Hi! What’s up?",
      "Hey! How’s your day going?",
      "Hello there!",
      "How's everything going today?",
      "Hi, what’s new?",
      "Hey! What's happening?",
      "Hello! How are things?",
      "Hey! Ready to chat?",
      "Hi! What's cracking?",
      "Hey! How's your week been?",
      "What’s the latest?",
      "Hi! Everything good?",
      "Hey! It’s been a while.",
      "Hello! How’s it going on your end?",
      "Hey! Anything exciting happening?",
      "Hi! What’s been good?",
      "Hey, got a minute?",
      "Hello! Any news from your side?",
      "Hi! How's your day shaping up?",
      "Hey! What are you up to?",
      "Hello! What’s been happening?",
      "Hi! Got any updates?",
      "What’s shaking?",
      "Hey! Good to see you online.",
      "Hi! Finally caught you!",
      "Hello! How was your day?",
      "Hey, got some time to chat?",
      "Hi! What's the buzz?",
      "Hey! How's life?",
      "Hello! All good on your side?",
      "Hey, how’s things?",
      "Hi! What’s new and exciting?",
      "Hey! Catch me up!",
      "Hello! How's your day rolling?",
      "Hi! Break time?",
      "Hey! Fill me in!",
      "What’s happening?",
      "Hello! Got any good stories?",
      "Hey! How’s your morning?",
      "Hi! What's your plan today?",
      "Hello! All’s well?",
      "Hey! Share a quick update?",
      "Hi! Been busy?",
      "Hey there! How's it hanging?",
      "Good day!",
      "Hi! Tell me something good!",
      "Hey! How have things been?",
      "Hello! What’s new with you?",
      "Hey! How’s everything?",
      "Hi! Hope you’re well!",
      "Hey! Give me the lowdown.",
      "Hello! What's the latest?",
      "Hi! Any updates from you?",
      "Hey! Anything new happening?",
      "Hi! How was your weekend?",
      "Hey! How’s everything going?",
      "Hello! How’ve you been?",
      "Hey! What's been going on?",
      "Hi! What’s your news?",
      "Hello! Any big plans today?",
      "Hey! Got any gossip?",
      "Hi! Hope it's a good day.",
      "Hey! What's up?",
      "Hello! What's new in your life?",
      "Hi! How’s your afternoon?",
      "Hey! And what’s been up?",
      "Hello! Tell me what’s new.",
      "Hi! Got any plans?",
      "Hey! Ready to chat a bit?",
      "Hello! What’s the good word?",
      "Hi! What’s the vibe today?",
      "Hey! What’s the haps?",
      "Hi! Fill me in!",
      "Hey! Ready to catch up?",
      "Hello! What's been good?",
      "Hi! Anything interesting today?",
      "Hey! How's the weather there?",
      "Hello! How’s your day unfolding?",
      "Hi! What's your status?",
      "Hey! What’s happening?",
      "Hello! Up for some news?",
      "Hi! What’s occurrin'?",
      "Hey! How’s your morning going?",
      "Hello! What’s the latest?",
      "Hi! Any exciting news?",
      "Hey! What have you been up to?",
      "Hey! Had any fun today?",
      "Hello! Everything alright?",
      "Hi! What’s new in your world?",
      "Hey! How’s it going today?",
      "Hello! Got any news to share?",
      "Hi! What's cooking?",
      "Hey! Give me a heads up.",
      "Hello! What’s up with you?",
      "Hi! How are you keeping?",
      "Hey! How was your night?",
      "Hello! What's the scoop?",
      "Hi! Any big plans this week?",
      "Hey! How’s everything with you?",
      "Hello! How's your weekend?",
      "Hi! What are you up to today?",
      "Hey! How have you been?",
      "Hello! Any buzz?",
      "Hi! What’s happening with you?",
      "Hey! What’s your latest adventure?",
      "Hello! How’s everything with you?",
      "Hi! What’s on your mind?",
      "Hey! Did you hear anything new?",
      "Hello! How’s your evening?",
      "Hi! Got anything to share?",
  ]
}


company = {
    "comments": [
        "What is the company culture like?",
        "I'm interested in the customer service role, can you send me more info?",
        "Do you offer any training programs for new hires?",
        "What are the working hours for the inbound sales team?",
        "How many days off do we get per year? 🤔",
        "Is there opportunity for growth within the company?",
        "What kind of benefits do you offer to employees?",
        "I'm interested in the data entry role, what are the responsibilities?",
        "Do you have a referral program for employees?",
        "What is the average salary for a customer service rep?",
        "Do you have a dress code policy?",
        "How does the performance evaluation process work?",
        "What kind of support does the company offer for professional development?",
        "Can you tell me more about the team I'll be working with?",
        "What are the company's short-term and long-term goals?",
        "Are there any opportunities for remote work?",
        "What's the typical career path for someone in the customer service team?",
        "Do you offer any employee discounts or perks?",
        "How does the company approach work-life balance?",
        "What sets this company apart from other call centers?",
        "What's the company's policy on flexible scheduling?",
        "Hey there! Could you tell me more about the company's history?",
        "Good morning! What are the company's main products or services?",
        "Hi! Can you provide some insights into the company culture?",
        "Hello! How does the company foster employee growth and development?",
        "Yo! What's the company's mission and values?",
        "Hey! I'm curious, what sets this company apart from its competitors?",
        "Hi! What's the company's stance on sustainability?",
        "Hey, what's the work environment like at the company?",
        "Hey! What's the company's approach to diversity and inclusion?",
        "Hi! Can you tell me about the company's financial performance?",
        "Hello! How does the company give back to the community?",
        "Hey, what's the average tenure of employees at the company?",
        "Hi! How does the company ensure a safe and inclusive workplace?",
        "Hey! Can you share any upcoming projects or initiatives the company is working on?",
        "Hello! What's the company's policy on remote work?",
        "Hi! How does the company encourage innovation among its employees?",
        "Hey! Can you tell me about the company's customer base?",
        "Hello! What training and development opportunities does the company provide?",
        "Hey, what benefits does the company offer to its employees?",
        "Hey! Can you provide some information about the company's technology stack?",
        "Hello! How does the company ensure data privacy and security?",
        "Hi! What's the company's policy on flexible working hours?",
        "Hey! I'm interested to know about the company's training programs.",
        "Hi! Can you tell me about the company's customer satisfaction ratings?",
        "Hey, what opportunities for professional development does the company offer?",
        "Hello! How does the company celebrate employee achievements?",
        "Hi! What's the company's approach to employee wellness?",
        "Hey! Can you share any success stories or case studies?",
        "Hello! How does the company support employee mental health?",
        "Hi! What's the company's strategy for growth and expansion?",
        "Can you tell me more about the company's culture?",
        "What's the typical career path for someone in this role?",
        "How do you measure success in this position?",
        "What are the main challenges new hires usually face?",
        "Is there room for growth within the company?",
        "What kind of training programs do you offer?",
        "Can you explain the team structure here?",
        "How do you handle employee feedback?",
        "What's the most rewarding part of working here?",
        "What are the shift patterns like?",
        "Do you offer remote working options?",
        "How do you support work-life balance?",
        "What are the next steps in the hiring process?",
        "Can you describe a typical day in this job?",
        "How does this role contribute to the company's goals?",
        "What's the company's stance on overtime?",
        "Do you provide any mental health support for employees?",
        "How often do you conduct performance reviews?",
        "What's the team dynamic like?",
        "Are there opportunities for advancement?",
        "What's the company's turnover rate?",
        "Can you tell me more about the company’s mission?",
        "How does the company recognize and reward outstanding performance?",
        "What are the biggest challenges the company is currently facing?",
        "How do you handle conflict within the team?",
        "What's the onboarding process like?",
        "How are goals and expectations communicated to the team?",
        "What kind of feedback can I expect from my supervisor?",
        "What's your policy on work-from-home?",
        "How diverse is your team?",
        "Are there opportunities for cross-training or job rotation?",
        "What tools and technologies will I be using?",
        "How does the company support professional development?",
        "What's the company’s approach to customer service?",
        "Do you have a mentoring program?",
        "What are the main goals for the team this year?",
        "Can you tell me about a successful project the team has completed recently?",
        "What type of clients or customers does the company serve?",
        "How does the company handle high-stress periods?",
        "What's the most common reason employees stay long-term?",
        "Do you offer any tuition reimbursement programs?",
        "How would you describe the company’s management style?",
        "What are the key skills needed for success in this role?",
        "What are the company's values?",
        "How do you ensure employees feel valued?",
        "Is there a dress code?",
        "How soon can I expect to hear back after this interview?",
        "What are the benefits and perks?",
        "How do you keep employees motivated and engaged?",
        "Do you celebrate achievements or milestones?",
      "What's the dress code like in the office?",
      "What are the main challenges the company is facing right now?",
      "How long has the company been in business?",
      "What kind of growth opportunities does the company offer?",
      "How does the company support professional development?",
      "What's the typical career path in this department?",
      "Does the company encourage teamwork?",
      "How often does the company conduct performance reviews?",
      "Are there opportunities for remote work?",
      "What kind of technology stack does the company use?",
      "What's the onboarding process like?",
      "How does the company foster innovation?",
      "Does the company have any plans for expansion?",
      "How does the company handle work-life balance?",
      "Are there any employee resource groups?",
      "What are the core values of the company?",
      "Can you tell me about a successful project the company recently completed?",
      "What industry does the company primarily operate in?",
      "What kind of training programs are available to employees?",
      "How does the company approach diversity and inclusion?",
      "What are the most important qualities you look for in a team member?",
      "Does the company offer internships or mentorship programs?",
      "What kind of feedback mechanism is in place for employees?",
      "How does the company reward performance?",
      "What benefits does the company offer?",
      "Are there any upcoming major projects or initiatives?",
      "How does the company measure success?",
      "Does the company organize team-building activities?",
      "What is the company's stance on continuous improvement?",
      "Are there any community engagement or CSR activities?",
      "Can you describe the leadership style of the company?",
      "What are the primary products or services the company offers?",
      "How is the workflow structured in the team?",
      "What's the company's approach to employee wellness?",
      "Are there any recent accomplishments the company is proud of?",
      "How open is the company to new ideas from employees?",
      "What kind of clients does the company work with?",
      "Can you give me an overview of the company's strategic goals?",
      "What's the most exciting thing about working here?",
      "Does the company encourage continuing education?",
      "How does the company handle conflict resolution?",
      "What software tools does the company use?",
      "What is the company's vision for the future?",
      "How are promotions decided?",
      "Are there opportunities for lateral moves within the company?",
      "How does the company ensure job satisfaction?",
      "What's the company policy on overtime?",
      "What sustainability initiatives does the company have?",
      "How does the company communicate its goals to employees?",
      "What's the management philosophy here?",
      "What kind of social events does the company hold?",
      "Are there opportunities to work on cross-functional teams?",
      "What’s the most challenging aspect of working for this company?",
      "How does the company support its community?",
      "Are there any notable partnerships or collaborations?",
      "How often do team members meet with their managers?",
      "What’s the procedure for internal job transfers?",
      "What’s the company's vision for digital transformation?",
      "How does the company address employee suggestions?",
      "Are there any opportunities for international assignments?",
      "What’s the company’s policy on workplace flexibility?",
      "Do you have annual company retreats?",
      "How does the company keep employees motivated?",
      "What’s the typical decision-making process?",
      "How are company goals communicated across different departments?",
      "Does the company support any charitable causes?",
      "Can you tell me about the company’s history?",
      "How does the company manage workloads?",
      "What’s a day in the life of an employee like?",
      "What steps does the company take to maintain a healthy work environment?",
      "What platforms does the company use for team communication?",
      "How transparent is the company about its financial performance?",
      "What’s the staff turnover rate?",
      "Can you describe the company’s approach to risk management?",
      "Are there opportunities for part-time work?",
      "How are employee achievements recognized?",
      "What’s the company’s approach to cybersecurity?",
      "How does the company support newly hired staff?",
      "What's the level of formality in daily communications?",
      "How does the company handle high-pressure situations?",
      "Can you describe the talent acquisition strategy?",
      "Does the company offer stock options or other incentives?",
      "Has the company received any awards or accolades?",
      "What’s the company’s strategy for customer engagement?",
      "What’s the role of technology in achieving the company’s business goals?",
      "How does the company foster a sense of community among its employees?",
      "Does the company have an employee recognition program?",
      "What’s the typical career progression here?",
      "How does the company approach client relationships?",
      "What’s the company's stance on remote work longevity?",
      "Are there opportunities for cross-departmental training?",
      "How accessible are higher management layers to employees?",
      "What does the company do to support personal growth?",
      "Does the company provide any financial planning resources?",
      "How does the company celebrate milestones?",
      "What sets the company apart from its competitors?",
      "How are work schedules typically structured?",
      "What’s the company policy on travel for work?",
      "How does the company stay ahead in its industry?",
      "Are there any unique perks offered to employees?",
      "How important is collaboration within the company?",
      "What’s the average team size?",
      "Do you partner with any industry associations?",
      "What’s the frequency of company-wide updates?",
      "How receptive is the company to feedback from clients?",
      "Are there any long-standing traditions or rituals here?",
      "What are the company’s primary revenue streams?",
      "How does the company keep up with market trends?",
      "What role does sustainability play in the company’s strategy?",
      "What professional backgrounds do most employees have?",
      "How does the company ensure compliance with regulations?",
      "What’s the company’s stance on social media use?",
      "Are there opportunities to present at industry conferences?",
      "How does the company approach employee bonuses?",
      "What’s the process for submitting a new idea?",
      "Do employees have a say in project selections?",
      "Are there opportunities to lead projects early in one’s career?",
      "How are conflicts within teams generally resolved?",
      "What continuing education opportunities are offered?",
      "What’s the role of mentorship here?",
      "Are there any upcoming changes in company policy?",
      "How does the company keep employees informed about its performance?",
      "What's the importance of networking within the company?",
      "How does the company engage with its alumni?",
      "Is there a probationary period for new employees?",
      "How does the company handle underperformance?",
      "What’s the process for project approvals?",
      "Are there any skills in particular the company values?",
      "How does the company balance short-term goals with long-term vision?",
      "Does the company use Agile methodologies?",
      "Are there opportunities to participate in hackathons or similar events?",
      "What’s the company’s philosophy on work hours?",
      "How does the company define and measure employee engagement?",
      "What’s the company’s track record with promoting internally?",
      "Does the company have a formal code of ethics?",
      "How are external partnerships managed?",
      "What’s the approach to cross-training employees?",
      "Are there co-working spaces available within the office?",
      "What’s the company's roadmap for future products?",
      "Does the company use data analytics for decision-making?",
      "What sets this company apart in terms of innovation?",
      "How frequent are one-on-one meetings with managers?",
      "Are there dress-down days or similar events?",
      "What are some key performance indicators for employees?",
      "How is success celebrated within the teams?",
      "What’s the retention rate for new hires?",
      "How does the company invest in new technologies?",
      "Are there opportunities for rotations across different roles?",
      "What types of problems is the company passionate about solving?",
      "How is employee feedback gathered and implemented?",
      "Are there any wellness programs for employees?",
      "How does the company handle competitive pressures?",
      "Are there set times for team collaboration?",
      "How quickly does the company adapt to change?",
      "Are there shared recreational spaces in the office?",
      "What’s the company's approach to intellectual property?",
      "Are there opportunities for community outreach?",
      "How does the company ensure its vision is shared by all employees?",
      "Does the company support esports or other leisure activities?",
      "How is the company’s work environment described by current employees?",
      "What are the strategic priorities for the coming years?",
      "Are there any diversity and inclusion initiatives?",
      "What steps does the company take to protect employee data?",
      "Are hybrid meetings a norm here?",
      "What learning resources are available for free?",
      "What drives the company’s success?",
      "Are there events like 'Lunch and Learn'?",
      "What’s the level of autonomy given to teams?",
        
    ]
}


feedbacks = {
    "feedback_requests": [
        "Can you please update me on my application status?",
        "What's the current stage of my hiring process?",
        "Feedback on my interview would be appreciated, thanks!",
        "How's my application doing? 🤔",
        "Any news on my job application?",
        "Could you share an update on my candidacy?",
        "What's the latest on my application?",
        "Kindly inform me about my application progress.",
        "Can I get an update on my interview?",
        "I'd love to know where I stand in the process.",
        "Is there any update on my application status?",
        "Please let me know if I'm still in the running?",
        "Can you give me some feedback on my resume?",
        "What's the current status of my application?",
        "Any feedback on my performance during the interview?",
        "Can I get an update on the hiring process?",
        "I'm eagerly waiting to hear about my application.",
        "What's the decision on my application?",
        "Is my application still being considered?",
        "Am I still in the running for this position?",
        "Hey there! Just checking in to see if there's any update on my interview status?",
        "Good morning! I was wondering if there's any news on where I stand in the evaluation process.",
        "Hi! Can you provide me with an update on the status of my application, please?",
        "Hello! I hope you're doing well. Could you please let me know the current status of my interview?",
        "Yo! Any chance I could get an update on how things are progressing with my evaluation?",
        "Hey! Just wanted to touch base and inquire about the status of my application.",
        "Hi! I hope all is going smoothly. Any updates on my interview status?",
        "Hello! Hope you're having a great day. Can you give me an update on my evaluation status?",
        "Hey there! Wondering if there's any news on my application status?",
        "Good morning! I hope you're doing well. Could you provide me with an update on my interview status?",
        "Hi! Just checking in to see if there's any progress with my evaluation.",
        "Hello! Hope you're having a good day. Any updates on my interview status?",
        "Hey! Any chance I could get an update on my application status?",
        "Hi! I hope you're doing well. Can you please let me know the status of my evaluation?",
        "Hello! Just reaching out to see if there's any update on my interview status.",
        "Hey there! Hoping to get an update on my application status.",
        "Hi! I'm eager to know the status of my evaluation. Any updates?",
        "Hello! Can you please provide me with an update on my interview status?",
        "Hey! Just wanted to follow up on my application status. Any news?",
        "What's the update on my application?",
        "Can you tell me where my evaluation stands?",
        "What's the current status of my interview process?",
        "How's my file looking?",
        "Any news on the selection process?",
        "Where are we at with the flow?",
        "What's the holdup?",
        "Can I get an update on the review process?",
        "How much longer til I hear back?",
        "What's the ETA on the next steps?",
        "Can someone give me a status update?",
        "What's the progress report?",
        "Am I close to getting an offer?",
        "What's the deal with my application?",
        "Can I get some clarity on the evaluation process?",
        "How's the process going?",
        "When can I expect to hear back?",
        "What's taking so long?",
        "Can you shed some light on the current status?",
        "What's the most recent update?",
        "How much longer do I have to wait?",
        "Can you walk me through the next steps?",
        "What's the latest on my evaluation?",
        "What's the plan moving forward?",
        "Can someone give me a heads up on the timeline?",
        "What's the status of the review?",
        "Am I still in the running?",
        "What's the hold up?",
        "Can I get a quick update?",
        "What's going on with the process?",
        "How's the team doing on the evaluation?",
        "When can I expect to hear some news?",
        "What's the word on my application?",
        "Can someone give me a status report?",
        "What's the current state of affairs?",
        "How's the flow going?",
        "What's the latest development?",
        "Can you give me a progress report?",
        "What's the deal with the delay?",
        "Am I still being considered?",
        "What's the status of the interview process?",

      "Could you please provide an update on my application status?",
      "Hey, what's the latest on my interview?",
      "Just checking in to see if there's any news about my job application.",
      "Can I get an update on my progress through the hiring process?",
      "Hi, any updates regarding my evaluation results?",
      "Hey there! Any word on my interview status?",
      "I'd like to know the current status of my job application.",
      "Could you let me know where I stand in the hiring process?",
      "Hi, how's my application coming along?",
      "Any news on my job application status?",
      "Looking for information on my progress in the hiring process.",
      "Hi, when can I expect to hear back about my interview?",
      "Is there any update on my application?",
      "Can you tell me how far along I am in the evaluation process?",
      "Hi, any feedback from my recent interview?",
      "Could you provide an update on my application review?",
      "Wanted to check if there are any updates on my job application.",
      "Hi, where do I currently stand in the interview process?",
      "Can you give me an update on my interview results?",
      "Just following up on the status of my job application.",
      "Could you please give me an update on my interview status?",
      "Hi, any updates from the hiring team?",
      "What's the status of my job application?",
      "Checking in on my job application status.",
      "Hello, can you update me on my evaluation?",
      "What's the latest update on my interview?",
      "Hey, is there any update on my job application?",
      "Can I get a status update on my application?",
      "Hi, any progress on my job application?",
      "Would it be possible to get an update on my interview?",
      "Looking for a status update on my job application.",
      "Hi, any news on my interview status?",
      "Could you provide me with an update on my evaluation?",
      "Hey, any updates on my application status?",
      "I'd like to know the current status of my interview.",
      "Hi, is there any news on my job application?",
      "Can you update me on my progress in the interview process?",
      "Hi, any news from the hiring team?",
      "What's the latest on my job application?",
      "Can I have an update on my evaluation status?",
      "Hello, any feedback on my interview?",
      "Hi there, what's the status of my application?",
      "Just checking in to see if there's an update on my interview.",
      "Any updates on where I stand in the hiring process?",
      "Could you please update me on my application status?",
      "Hi, where do I stand in the job application process?",
      "Hey, can I get an update on my evaluation?",
      "What's the current status of my interview?",
      "Hi, any news on my evaluation status?",
      "Could you provide an update on my interview feedback?",
      "Just wanted to follow up on my job application status.",
      "Can you let me know the status of my application?",
      "Hi, any progress on my interview?",
      "I'd like an update on my evaluation status.",
      "Could you please let me know the status of my job application?",
      "Hey, any update on my interview feedback?",
      "Hi, any news on my application's progress?",
      "Can you provide an update on my evaluation results?",
      "I'd appreciate an update on the status of my interview.",
      "Could you let me know if there are any updates on my job application?",
      "Hello, any feedback on my evaluation?",
      "Hi, can you tell me the status of my interview?",
      "Just checking in to see if there are any updates on my application.",
      "Could you provide me with an update on my job application status?",
      "Hi, what's the latest on my interview status?",
      "Any updates on my job application's progress?",
      "Can I get an update on my interview feedback?",
      "Hello, any news on my evaluation?",
      "Hi, where do I stand with my job application?",
      "Just checking in on my application status.",
      "Could you please update me on my interview results?",
      "Hey, any updates on my job application's status?",
      "Hi, any feedback from my evaluation?",
      "Can you tell me the status of my application?",
      "What's the current status of my evaluation?",
      "Hi, any news on my interview feedback?",
      "Could you provide an update on my job application's progress?",
      "Just following up to see if there are any updates on my evaluation.",
      "Any updates on my job application status?",
      "Hi, is there any news on my interview feedback?",
      "Can I have an update on my evaluation results?",
      "What's the latest on my job application status?",
      "Hi, any progress on my evaluation feedback?",
      "Could you please give me an update on my interview results?",
      "Just checking in to see if there's any news on my evaluation.",
      "Hi, where do I currently stand with my job application?",
      "Any news on my job application's status?",
      "Hi, any updates from the hiring team on my evaluation?",
      "Hello, what's the status of my job application?",
      "Just following up to see if there are any updates on my interview.",
      "Hi, any feedback on my evaluation status?",
      "Could you provide me with an update on my application progress?",
      "Hey, any word on my job application's status?",
      "Can I get an update on my application progress?",
      "Hi, any news on my interview results?",
      "Just checking in on my application's status.",
      "Could you please update me on my evaluation feedback?",
      "Hi, where do I stand in the hiring process?",
      "Any updates on my interview progress?",
      "Hi, any updates on my job application?",
      "Could you provide an update on my evaluation status?",
      "Just following up to see if there's any news on my application.",
      "Hey, what's the latest on my evaluation feedback?",
      "Hi, any news on my job application's progress?",
      "Can you update me on my evaluation results?",
      "What's the current status of my application?",
      "Hi, any updates from the hiring team on my interview?",
      "Could you provide an update on my job application status?",
      "Hey, any word on my evaluation results?",
      "Hi, where do I stand with my evaluation?",
      "Just checking in to see if there's any news on my interview.",
      "Hi, any feedback on my job application?",
      "Could you please update me on my application's progress?",
      "What's the status of my evaluation?",
      "Hi, any news on my evaluation feedback?",
      "Can you tell me the status of my job application?",
      "Hey, what's the latest on my job application?",
      "Just following up to see if there are any updates on my interview feedback.",
      "Hi, where do I currently stand with my evaluation?",
      "Any updates on my job application's progress?",
      "Hi, any feedback from the hiring team on my application?",
      "Could you provide an update on my interview results?",
      "Hey, any word on my job application's progress?",
      "Just checking in on my evaluation feedback.",
      "What's the current status of my job application?",
      "Hi, any updates on my evaluation feedback?",
      "Can you update me on my interview feedback?",
      "Could you please give me an update on my job application?",
      "Hello, any news on the status of my evaluation?",
      "Hi there, where do I stand in the job application process?",
      "Any updates on my evaluation feedback?",
      "Hi, can you provide an update on my interview feedback?",
      "Just checking in to see if there's any news on my evaluation status.",
      "Hey, what's the latest on my job application's status?",
      "Any feedback from the hiring team on my evaluation?",
      "Hi, what's the status of my evaluation process?",
      "Any updates on my interview feedback?",
      "Just following up to see if there are any updates on my job application.",
      "Hi, where do I stand with my interview status?",
      "Hey, any updates on my application progress?",
      "What's the current status of my evaluation feedback?",
      "Could you provide an update on my job application progress?",
      "Just checking in to see if there's any news on my interview status.",
      "Hi, any feedback on my evaluation?",
      "Any updates from the hiring team on my interview?",
      "Hi, what's the status of my application?",
      "Hey, any word on my evaluation feedback?",
      "Just following up to see if there are any updates on my application status.",
      "Any updates on my job application feedback?",
      "Hi, where do I stand with my evaluation status?",
      "Can you provide an update on my interview feedback?",
      "Just checking in on my evaluation's progress.",
      "Could you please give me an update on my interview feedback?",
      "Hi, what's the current status of my job application?",
      "Any updates from the hiring team on my application status?",
      "Hey, any news on my application process?",
      "Could you update me on my evaluation results?",
        
    ]
}


continue_later = {
    "phrases": [
        "I'm taking a break, talk to you later on WhatsApp!",
        "I need to go, catch you later!",
        "I'm feeling a bit tired, will continue this convo later 😴",
        "I'm out for now, will get back to you later!",
        "Have to run, talk to you soon!",
        "I'm not feeling up to chat right now, will continue later",
        "I'm stepping away for a bit, be back soon!",
        "I'm bored with this convo, will catch you later",
        "I have to go, but we can pick this up later",
        "I'm not able to continue right now, will talk to you later",
        "I need some time to myself, talk to you later!",
        "I'm taking a time-out, will come back to this later",
        "I'm not feeling the energy to chat, will catch you later",
        "I have to go, but we can continue this on WhatsApp later",
        "I'm disconnecting for now, talk to you soon!",
        "I'm not able to focus, will pick this up later",
        "I'm out of energy, will catch you later!",
        "I need a break from this convo, will continue later",
        "I'm stepping away, but we can continue this on WhatsApp later",
        "I'm not able to chat right now, will talk to you soon!",
        "I'm feeling drained, will continue this later",
        "Hey there! Thanks for reaching out. I'm currently tied up with a meeting, can we continue this later?",
        "Good morning! I'm in the middle of something right now, can we pick this up in an hour?",
        "Hi! Sorry, I'm swamped at the moment. Can we chat about this tomorrow?",
        "Hello! I'm unable to continue the process at the moment due to some unforeseen circumstances.",
        "Yo! Can we put a pin on this for now? I'll get back to you as soon as I can.",
        "Hey! Unfortunately, I won't be able to proceed further with the interview today. Can we reschedule?",
        "Hi! I'm tied up with back-to-back meetings. Can we touch base later?",
        "Hello! I'm currently on the go, so I'll have to get back to you about this later.",
        "Hey there! I'm really sorry, but I have an urgent task to attend to. Can we resume this discussion tomorrow?",
        "Good morning! I'm running a bit behind schedule today. Can we continue this conversation later?",
        "Hi! I'm afraid I won't be able to proceed with the evaluation at the moment. Can we reconvene tomorrow?",
        "Hello! I'm currently dealing with an emergency situation. Can we postpone this to a later time?",
        "Hey! I'm really sorry, but I have to step away for a moment. Can we continue this in an hour?",
        "Hi! Unfortunately, I've hit a roadblock in the process. Can we revisit this tomorrow?",
        "Hey there! I'm in the middle of something important. Can we catch up on this later?",
        "Hello! I'm sorry, but I'm unable to dedicate time to this right now. Can we discuss this tomorrow?",
        "Hi! I'm currently occupied with a pressing matter. Can we pick this up later in the day?",
        "Hey! I'm juggling multiple tasks at the moment. Can we continue this at a more convenient time?",
        "Hi! I'm really sorry, but I have to step out unexpectedly. Can we continue this discussion later?",
        "Hello! I'm tied up with a deadline right now. Can we revisit this tomorrow?",
        "Can we catch up later? 😊",
        "I'm running behind, can we reschedule?",
        "I'm slammed today, can we talk later?",
        "I don't have time right now, sorry!",
        "I'm not available at the moment, can we schedule for later?",
        "Maybe later today? My schedule is crazy!",
        "I'm in the middle of something, can we talk soon?",
        "I'm on a deadline, can we catch up later?",
        "I'm tied up right now, can we talk later?",
        "I don't have a minute to spare, can we talk soon?",
        "Let's talk later, I'm swamped!",
        "I'm not free at the moment, can we schedule for later?",
        "I'm in a rush, can we catch up soon?",
        "I need some time, can we talk later?",
        "Can we chat later? I'm on a call!",
        "I'm running low on time, can we talk soon?",
        "I'm not available now, can we schedule for later?",
        "Maybe some other time? I'm busy!",
        "I'm on a tight deadline, can we talk later?",
        "I don't have the bandwidth for that right now!",
        "Let's talk later, I'm in a meeting!",
        "I'm in the middle of something, can we catch up soon?",
        "Not a good time for me, can we reschedule?",
        "I'm slammed with work, can we talk later?",
        "I need some time to focus, can we talk soon?",
        "I'm on a roll, can we catch up later?",
        "I'm not free to chat right now, sorry!",
        "Can we talk later? I'm in the middle of something!",
        "I'm running out of time, can we talk soon?",
        "I'm not available at the moment, can we schedule later?",
        "Maybe later today? I'm crazy busy!",
        "I'm tied up with work, can we catch up later?",
        "I don't have time to chat, sorry!",
        "Let's talk later, I'm on a deadline!",
        "I need some time to focus, can we talk later?",
        "Can we catch up soon? I'm slammed!",
        "I'm not free to chat right now, can we talk later?",

      "I'll need to pick this up later.",
      "Can we continue this another time?",
      "I'll have to come back to this.",
      "I can't finish this now, let's resume later.",
      "I'm tied up at the moment; let's chat later.",
      "Need to pause here, continue soon.",
      "Sorry, I have to go. Let's continue later.",
      "I'll get back to this later.",
      "I'll return to this task later on.",
      "We'll have to pick this up another time.",
      "I need to interrupt, let's finish later.",
      "Gotta run now; let's reconnect later.",
      "Can we circle back to this?",
      "I'm stepping away for now. Back later.",
      "Put a pin in it for now; follow up later.",
      "I'll handle this later when I have more time.",
      "I'll follow up with you later on this.",
      "I need to stop here, can we resume later?",
      "Let's put this on hold for now.",
      "I'm pausing for now, will continue later.",
      "Can we delay this to another time?",
      "I'll need to continue this conversation later.",
      "Can we catch up on this soon?",
      "I'll have to take care of this later.",
      "Sorry, I can't continue now. Can we reschedule?",
      "Can we revisit this later?",
      "I'm out of time, let's finish this later.",
      "I'll need to come back to this later.",
      "Can we take this up some other time?",
      "I'll resume this later.",
      "Gotta pause here, back later.",
      "Let's hit pause on this for now.",
      "I'll address this later when I'm free.",
      "I'm dropping off for now, will continue later.",
      "Let's pick this back up later.",
      "I have to stop here, can we continue later?",
      "Let's shelve this for the moment.",
      "I'll look into this later.",
      "Can we bookmark this for later?",
      "I'll revisit this when I can.",
      "We can resume this conversation later.",
      "I'll have to follow up later.",
      "I'm pausing here, continue soon.",
      "Let's postpone this till later.",
      "I'll check back on this later.",
      "Can we put this on hold for now?",
      "I'll return to this discussion later.",
      "Let’s defer this until later.",
      "I’m not able to continue now, we'll talk later.",
      "We need to reschedule this for later.",
      "I'll come back to this soon.",
      "Sorry, I have to leave now, continue later?",
      "I'm unable to complete this now, let's do it later.",
      "Can we handle this later?",
      "I'll get back to this when I have a minute.",
      "Catch you later to finish this.",
      "Need to pause, will continue in a bit.",
      "Let's leave this for now, get back later.",
      "I'll tackle this later.",
      "We'll need to finish this another time.",
      "This needs to wait, catch up later.",
      "I have to step out, resume later.",
      "Let's hold off on this for now.",
      "I’m stepping away, let's revisit later.",
      "I'll put this on my list to finish later.",
      "I’ll follow up once I have more time.",
      "Let’s pause here, continue soon.",
      "I'll need to delay this for now.",
      "We’ll pick this conversation up later.",
      "I can't do this now, let’s plan for later.",
      "Can we circle back to this later?",
      "I'll need to continue this at another time.",
      "Let’s reconnect later to finalize.",
      "Gotta break now, continue later.",
      "I'll need to address this when I'm free.",
      "We'll get back to this in a bit.",
      "Need to step away, let's resume later.",
      "I have to postpone this for now.",
      "I’ll look into this later on.",
      "Can we return to this conversation later?",
      "I need to put this on hold for now.",
      "I'll handle this when I have more time.",
      "Sorry, can't handle this now. Can we continue later?",
      "I'll touch base later on this.",
      "Need to break away, back soon.",
      "I'm unable to proceed right now, catch up later?",
      "I’ll have to revisit this later.",
      "Can we continue this at a later time?",
      "I'll pause here, continue shortly.",
      "Schedule this for later, need to step away.",
      "We'll come back to this later.",
      "Can we hit pause and resume later?",
      "We'll follow up on this later.",
      "I'll check back on this in a bit.",
      "I'll need to address this another time.",
      "Putting a pin in this, will continue later.",
      "Let’s table this for another time.",
      "I'll review this later on.",
      "I have to wrap up for now. We'll continue later.",
      "I need to reschedule this for later.",
      "We'll need to discuss this some other time.",
      "I need to get back to this later.",
      "Can we pick this up at another time?",
      "I'll follow up on this later.",
      "Need to halt here, continue later.",
      "Gotta jet now, finish later?",
      "I can't continue now, let's resume later.",
      "Can we handle this some other time?",
      "I'm taking a break, continue soon.",
      "Let's save this for later.",
      "I'll finish this up another time.",
      "I need to put a pause on this. Catch up shortly.",
      "I need a break, let's get back to this.",
      "We can handle this later.",
      "I’ll continue this soon.",
      "Can we revisit this soon?",
      "I can't manage this right now, let's do it later.",
      "I'll follow up later when I can.",
      "I'll circle back to this.",
      "Let's come back to this when possible.",
      "I need to defer this conversation.",
      "I’ll need to put this on the back burner.",
      "Let’s put a pin in this, we’ll resume later.",
      "Can we address this later?",
      "I need to pause here, we'll continue shortly.",
      "I’ll catch up on this later.",
      "I'll need to break here and continue later.",
      "Can we resume this shortly?",
      "I have to step away, we’ll continue later.",
      "I'm stopping now, continue later?",
      "I'll review this when I can.",
      "We need to finish this another time.",
      "I need to put this aside for now.",
      "I will address this later when I can.",
      "Let's deal with this later.",
      "Gotta take off now, continue the talk later.",
      "I can’t follow through now, will do later.",
      "Can we break and continue later?",
      "I’ll step back and revisit this later.",
      "I'll pause this for now.",
      "We'll get back to this conversation.",
      "I gotta bounce, catch up later?",
      "Let’s proceed with this another time.",
      "I'll need to touch base later.",
      "I gotta pause, finish up later.",
      "I'll reach out later to continue.",
      "I'm wrapping this up for now.",
      "I need to leave, we'll chat later.",
      "Let's delay the rest till later.",
      "Will reconnect soon to continue.",
      "I'll check back soon on this.",
      "We should take this up later.",
      "I’ll need to get back to you on this.",
      "I’m tapping out now, back soon.",
      "I'll revisit shortly.",
      "I gotta step away, let’s finalize later.",
      "I'll follow up when I can.",
      "Let's hold this topic for later.",
    ]
}


stop_continue = {
    "responses": [
        "Thanks for the offer, but I’ll pass this time.",
        "I appreciate it, but I’m not interested in continuing.",
        "I'm going to have to decline, thanks for considering me.",
        "Sorry, but I don’t agree with this approach.",
        "I’ve decided not to move forward with this process.",
        "Thanks, but I’m not up for it.",
        "Unfortunately, I have to say no.",
        "I’m not comfortable with this, sorry.",
        "I’m out, but thanks for the opportunity.",
        "I’ve thought about it, and I’m not interested.",
        "Thanks, but this isn't for me.",
        "I'm going to step back from this, but thank you.",
        "I’m choosing not to proceed, thanks.",
        "This doesn't align with my interests, so I’m not continuing.",
        "I’ll be bowing out of this process.",
        "I’m going to pass on this, thanks though.",
        "No, I’m not going to accept this.",
        "I can’t go along with this plan.",
        "I have to say no, but I appreciate it.",
        "Not agreeing with this, sorry.",
        "I’m not interested in pursuing this further.",
        "I’m going to have to pass on this opportunity.",
        "I won’t be moving forward with this, thanks.",
        "This isn’t something I want to do, sorry.",
        "I’m not on board with this.",
        "Nope, not for me. Thanks anyway!",
        "I’m not feeling this, so I’m out.",
        "I’m going to decline, but I appreciate the offer.",
        "This doesn’t work for me, so I’m saying no.",
        "I’ve decided against this, thanks anyway.",
        "I’m not continuing with this, sorry.",
        "I’m declining to move forward, thanks.",
        "Thanks, but I’ll have to decline.",
        "This isn't the right fit for me, so no.",
        "I don't agree with this, so I won't proceed.",
        "I’m going to have to say no to this.",
        "Sorry, but I’m not interested in this.",
        "Thanks, but I’m not going ahead with this.",
        "I’m passing on this, but thanks.",
        "I’m choosing not to participate, thank you.",
        "I'm not interested, sorry!",
        "I don't think so!",
        "Not for me, thanks!",
        "I'm out!",
        "I don't agree!",
        "Not my thing!",
        "No way, sorry!",
        "I'm not on board with that!",
        "I'll have to pass!",
        "Not interested!",
        "I don't want to continue!",
        "I'm not down for that!",
        "I'm good, thanks!",
        "No thanks!",
        "I'll have to say no!",
        "That's not for me!",
        "I'm not into that!",
        "I don't want to do that!",
        "I'm not up for it!",
        "Not my style!",
        "I'm not feeling it!",
        "I'll have to decline!",
        "I don't think that's for me!",
        "Nope, not interested!",
        "I'm not sure I want to!",
        "I'm not ready for that!",
        "I'll have to say no, thanks!",
        "That doesn't work for me!",
        "I'm not comfortable with that!",
        "I don't think I can do that!",
        "Nah, I'm good!",
        "I'll have to pass, sorry!",
        "I don't want to go further!",
        "I'm not on the same page!",
        "I don't agree with that!",
        "I'll have to say no way!",
        "I don't think that's a good idea!",
        "No thanks, not for me!",
        "I'll have to decline, thanks!",
        "I'm not up for that, sorry!",
        "That's not something I want to do!",
        "I don't want to proceed!",
        "I'll have to say no, sorry!",
        "I'm not interested, thanks!",

      "I think I'll pass on this opportunity.",
      "I'm not interested, thanks.",
      "This isn't the right fit for me.",
      "I’m not sure this aligns with my goals.",
      "Sorry, but I'm going to decline.",
      "I don't see this working out for me.",
      "I’ll have to say no to this.",
      "Not quite what I'm looking for.",
      "I'm not feeling this, sorry.",
      "I'm afraid I have to opt out.",
      "After consideration, I must decline.",
      "This job is not what I'm looking for.",
      "I'm sorry, but I can't continue.",
      "I'm going to withdraw my application.",
      "This isn't suitable for my needs.",
      "I've decided not to go further.",
      "I don’t think this is for me.",
      "I'm stepping back from this process.",
      "This doesn't match my expectations.",
      "I'm not convinced it's a good fit.",
      "I'll have to take a pass.",
      "Perhaps this isn’t my path.",
      "I can't commit to this.",
      "Unfortunately, I must decline.",
      "This just isn't for me.",
      "No thanks, I'll pass.",
      "I think I'll sit this one out.",
      "This isn't what I'm after.",
      "I'm not on board with this.",
      "This doesn’t seem right for me.",
      "I’ll bow out now.",
      "I don’t think this suits me.",
      "Count me out, please.",
      "I have to decline.",
      "This isn't something I agree with.",
      "I'm going to say no.",
      "It's not working for me.",
      "Sorry, but I'll have to withdraw.",
      "Not my cup of tea.",
      "I'm not going ahead with this.",
      "This isn’t aligning with my goals.",
      "Just not interested, thanks.",
      "I won't be moving forward.",
      "I have decided not to proceed.",
      "I don't think I can accept this.",
      "I'm out, sorry.",
      "No thank you, I’m not interested.",
      "This isn't what I want.",
      "I can't see myself fitting here.",
      "I must reject this offer.",
      "This isn't my thing.",
      "This doesn't appeal to me.",
      "I can't do this, sorry.",
      "I'm not continuing with this.",
      "Doesn't work for me, thanks.",
      "Sorry, but I have to decline.",
      "I can't see myself in this role.",
      "Nope, this isn't for me.",
      "I won't be accepting this.",
      "I must decline the offer.",
      "I don’t think this is where I belong.",
      "This is not what I envisioned.",
      "I'm not comfortable with proceeding.",
      "I won't be taking this up.",
      "This doesn't resonate with me.",
      "I find myself not interested.",
      "This isn't in my plans.",
      "I'm walking away from this one.",
      "I'm not signed up for this.",
      "I respectfully decline.",
      "I don't agree with this direction.",
      "I’m not buying this.",
      "Thanks but no thanks.",
      "I can't see this happening.",
      "I'm not okay with this.",
      "I’m gonna pass on this.",
      "I’m not feeling it.",
      "It's a no from me.",
      "I'm gonna skip this one.",
      "This isn't aligning with my interests.",
      "Just not what I want.",
      "I don't agree with this approach.",
      "I’ll step away now.",
      "I must decline this opportunity.",
      "I don't see myself in this process.",
      "I can't go along with this.",
      "I'm afraid it's not for me.",
      "No, this isn't right.",
      "I won't proceed any further.",
      "I'm not inclined to accept.",
      "I'm saying no to this.",
      "I don’t support this.",
      "This doesn’t float my boat.",
      "I’m outta here.",
      "Declining respectfully.",
      "I don't think this suits me well.",
      "I can't move forward with this.",
      "I'm leaving this path.",
      "This isn't my priority.",
      "Sorry, but this isn't right.",
      "Just not the right thing for me.",
      "Not interested in continuing.",
      "Don't want to go this way.",
      "I'm not the one for this.",
      "This is not up my alley.",
      "This is not appealing to me.",
      "I don't see myself fit here.",
      "No, I'm good.",
      "Not buying into this.",
      "Count me out.",
      "I don't align with this.",
      "Pass, thanks.",
      "This doesn't work for me.",
      "This isn't something I can accept.",
      "It’s a no.",
      "I'm gonna bounce on this.",
      "No can do.",
      "This isn't up my street.",
      "I can't say yes to this.",
      "I'm declining this offer.",
      "This isn't what I expected.",
      "Not my thing.",
      "I won't be following up.",
      "I choose not to continue.",
      "Sorry but I gotta pass.",
      "I’m not vibing with this.",
      "No go for me.",
      "Do not interest me.",
      "Nope, not doing it.",
      "Not gonna happen.",
      "This isn't aligning with me.",
      "I'm stepping away.",
      "This is not my direction.",
      "I'm not going to continue.",
    ]
}


trouble = {
    "responses": [
    "I'm struggling to understand this step, can someone help?",
    "This part of the process is confusing, assistance needed.",
    "Can someone guide me through this section?",
    "I'm having trouble with this. Any help would be appreciated.",
    "I can't figure this out. Can someone assist me please?",
    "This part doesn't make any sense to me. Help!",
    "I'm stuck here, how do I move forward?",
    "Can someone explain this to me? I'm lost.",
    "I'm not getting this. Could someone assist?",
    "I need help with this section, it's really confusing.",
    "I'm facing issues with this step. Can anyone help?",
    "Can anyone give me a hand with this part?",
    "This step is too hard, I'm struggling 😕",
    "I'm lost, can someone guide me?",
    "I'm having a really hard time with this, any assistance?",
    "Help! I can't understand this.",
    "This is really confusing, can someone help me out?",
    "I need some help, this section is too complicated.",
    "I'm finding this part difficult. Can anyone assist?",
    "I'm stuck at this point, need help urgently.",
    "Can someone clarify this step for me?",
    "I'm not sure what to do here, can someone assist?",
    "This part is tricky, can anyone help?",
    "I'm having a lot of trouble understanding this, help!",
    "I'm really confused, can somebody help me?",
    "I can't make sense of this. Could anyone assist?",
    "I'm struggling to move past this, assistance needed.",
    "This step is too complicated, help required.",
    "Can someone help me figure this out?",
    "I'm really stuck here, can anyone assist?",
    "I need help understanding this part.",
    "I'm having difficulty with this. Could someone help?",
    "Can someone guide me on what to do next?",
    "This is difficult, I need some help.",
    "I'm not following this, can someone assist me?",
    "This step is unclear, can anybody help?",
    "Help, I'm stuck on this part.",
    "Can someone explain this to me please?",
    "I'm lost, need assistance.",
    "I'm having issues with this step, can anyone lend a hand?",
    "Finding this really hard, any help?",
    "I'm not getting this step. Can someone help?",
    "Can anyone assist me with this part?",
    "I'm struggling, can anyone offer assistance?",
    "This is confusing, need help.",
    "Can't understand this part, can someone explain?",
    "I'm lost on this, need help.",
    "This isn't making sense, can anyone help?",
    "Having trouble here, need some help.",
    "Really struggling with this, can someone assist?",
    "I'm finding this part difficult, could anyone help?",
    "Need guidance with this step, assistance required.",
    "Can somebody explain this to me?",
    "This part is tough, help appreciated.",
    "I'm not sure how to proceed, can someone guide me?",
    "Struggling to understand this, any help?",
    "I can't get past this part, help needed.",
    "This is hard, can someone assist?",
    "This step is difficult, can anyone help?",
    "Having a tough time understanding this, need help.",
    "I'm confused, is there anyone who can help?",
    "I need a hand with this part, can someone assist?",
    "Stuck on this, can somebody help me out?",
    "I'm having a lot of trouble with this section, any help appreciated.",
    "This part is really tricky, can anyone guide me?",
    "Can't seem to understand this part, can someone help?",
    "I need help moving forward, this part is confusing.",
    "This step isn't clear, assist needed.",
    "I'm lost at this step, can someone help me?",
    "Need assistance with this, it's not clear.",
    "This is confusing, any help?",
    "Can someone direct me on this step?",
    "Struggling here, help required.",
    "I don't understand this part, can someone guide?",
    "Finding it hard to get through this, need help.",
    "Can't comprehend this step, any assistance?",
    "This step is bewildering, any help appreciated.",
    "I'm lost with this part, can someone help?",
    "Can someone provide guidance on this?",
    "I'm having difficulties, can anybody help?",
    "Struggling to get this, help needed.",
    "Confused by this step, need assistance.",
    "I'm blocked at this point, any help?",
    "Having issues here, can someone assist?",
    "This section is too complicated, any help?",
    "Need some guidance on this part, help needed.",
    "Having a hard time here, can someone assist?",
    "This is tough, could someone help me out?",
    "This step is confusing, can anyone lend a hand?",
    "Having trouble, need assistance.",
    "Can't seem to figure this out, any help?",
    "I'm confused by this, can someone assist?",
    "Need help, this part is challenging.",
    "Help! This is difficult for me.",
    "Can't make sense of this, need assistance.",
    "I'm having a hard time with this step, help.",
    "This is really difficult, can anyone help?",
    "I'm stuck on this part, any assistance?",
    "Can't get through this, need help.",
    "I'm lost, need some guidance.",
    "This is too complicated, any help?",
    "Having problems with this step, help appreciated.",
    "Frustrated with this, can someone assist?",
    "This is too hard, need help.",
    "Can't move forward, any help?",
    "This isn't clear to me, can someone assist?",
    "Help needed, this part is tough.",
    "I can't understand this, any help?",
    "Having difficulty, can someone guide me?",
    "This step confuses me, help needed.",
    "I'm stuck here, any assistance?",
    "I need help figuring this out.",
    "Can someone explain this section to me?",
    "Can someone assist me with this issue?",
    "This process is confusing, help needed.",
    "I'm having a difficult time, can someone help?",
    "This part is unclear, need help.",
    "Need guidance on this step, any assistance?",
    "Having trouble understanding this, help appreciated.",
    "I'm having a tough time here, need assistance.",
    "Can't figure out the next step, any help?",
    "I'm thoroughly confused, can someone help?",
    "This part is baffling, help needed.",
    "Need clarification on this step, any help?",
    "I'm stuck and need assistance.",
    "Not understanding this, any guidance?",
    "Can someone provide assistance with this step?",
    "I'm lost on this section, help needed.",
    "Need some help, this isn't clear.",
    "Having issues progressing, need assistance.",
    "This part is confusing me, help required.",
    "Sometimes 🤔, I don't get it, assistance?",
    "Stuck here, can someone help?",
    "Having problems with this, need help.",
    "I'm not following this part, can anyone guide me?",
    "This step is hard for me, need assistance.",
    "Can't make sense of this, help appreciated.",
    "I'm lost, any help would be great.",
    "I need some help, this step is confusing.",
    "This part is too tough, can someone assist?",
    "Gosh! I'm stuck here. Any help?",
    "I'm stuck. Someone please help?",
    "Difficulty moving ahead, need assistance.",
    "I'm having a real problem with this step, help needed.",
    "Can't progress. Any help?",
    "Can anyone guide me past this step?",
    "This is tough, can someone guide?",
    "Struggling with this, any help is welcome.",
    "I need assistance with this tough part.",
    "Having a problem here, can someone help?",
    "This part is difficult, help needed.",
    "Can't follow this, assistance required.",
    "This is unclear, can someone explain?",
    "Need some help here, this isn't making sense.",
    "This section is challenging, any help?",
    "Can't seem to move forward, need assistance.",
    "This part is really tough, any help?",
    "I'm having difficulty, can anyone assist?",
    "Need help with this part, it's confusing.",
    "This isn't making sense to me, help needed.",
    "Can someone help me with this confusing part?",
    "Having issues understanding this, any help?",
    "This step is very hard, need assistance.",
    "I can't figure this out, need help.",
    "Please assist me with this part.",
    "I'm totally lost, assistance needed.",
    "This process is confusing, can someone help?",
    "I'm having trouble getting through this step.",
    "I need help understanding this section.",
    "Struggling to figure this out, any help?",
    "Confused by this step, need guidance.",
    "I'm stuck, any help would be useful.",
    "Need assistance with this process, it's tough.",
    "This step is unclear, help required.",
    "Trouble moving ahead, can someone assist?",
    "Really confused here, need help.",
    "Stuck, can someone guide me?",
    "Can someone explain this part, it's confusing?",
    "This is challenging, can anyone assist?",
    "Can't get through this step, need help.",
    "Help required, I'm stuck here.",
    "This part is hard, can anyone lend assistance?",
    "Don't understand this, any help?",
    "Can't make sense of this process, help needed.",
    "Need some help, this is tricky.",
    "I'm lost, assistance appreciated.",
    "Need guidance through this step.",
    "Having trouble here, any help?",
    "This is difficult for me, any assistance?",
    "Can't figure out this step, need help.",
    "This step is confusing, need some guidance.",
    "Any help with this part would be great.",
    "Help, can't understand this step.",
    "I'm having a difficult time here.",
    "Need some guidance with this step.",
    "Can't get past this, help!",
    "This step is tough for me, need assistance.",
    "Help needed with this challenging part.",
    "I'm having issues, any help?",
    "Need someone to guide me through this.",
    "I'm stuck at this point, help required.",
    "This part is really hard, need some assistance.",
    "Confused by this step, can anyone help?",
    "Need clarification on this difficult part.",
    "Having difficulties here, help appreciated.",
    "Struggling with this, any assistance?",
    "Can't get through this, need some help.",
    "Having trouble with this confusing part.",
    "Lost! Need some guidance.",
    "Need some help with this part, can't figure it out.",
    "Can't see a way forward, help needed.",
    "I'm stuck here, any help appreciated.",
    "This part is really difficult, need clarification.",
    "Having issues moving forward, any help?",
    "Need some guidance here, having trouble.",
    "Can't make it past this step, assistance required.",
    "Help, I'm not understanding this process.",
    "This is so confusing, can anyone assist?",
    "Need assistance here, can't understand this step.",
    "Totally stuck, any help would be useful.",
    "Struggling with this part, help needed.",
    "Need guidance with this challenging step.",
    "I'm having trouble, any help appreciated.",
    "Can't get through this, need assistance.",
    "Having a lot of issues, can someone help?",
    "This step is really confusing, need assistance.",
    "Can anyone help me with this difficult part?",
    "Need clarification on this, having trouble.",
    "Need help with this confusing step, assistance appreciated.",

    ]
}


himself = {
    "response": [
      "I am highly proficient in troubleshooting technical issues quickly.",
      "Can integrate seamlessly into different cultural environments.",
      "I have advanced skills in various customer relationship management (CRM) software.",
      "Believe in maintaining a positive attitude even under pressure.",
      "I am always open to learning new sales techniques.",
      "Excel in providing excellent customer service through chat.",
      "I am motivated by reaching personal and collective goals.",
      "Adapt quickly to fast-paced environments and changing priorities.",
      "I can handle irate customers with patience and diplomacy.",
      "Value teamwork and collaboration in achieving targets.",
      "Aspire to lead my own team in the near future.",
      "I have a knack for understanding complex systems and processes.",
      "Enjoy contributing to a team that's focused on customer satisfaction.",
      "I have successfully managed high-volume chat sessions.",
      "Find joy in resolving customer issues efficiently.",
      "I aim to become an industry expert through continuous education.",
      "Mastered several programming languages relevant to tech support.",
      "I believe in fostering a company culture of inclusivity.",
      "Look forward to driving sales growth in my future roles.",
      "I can conduct thorough evaluations of customer needs.",
      "Take pride in providing detailed technical support.",
      "I am eager to advance my career in a dynamic environment.",
      "Have honed my interpersonal skills through diverse experiences.",
      "I am committed to lifelong learning and professional development.",
      "Can navigate multiple software platforms simultaneously.",
      "I believe in treating all customers with respect and empathy.",
      "Focus on achieving long-term career success.",
      "Known for my strong work ethic and reliability.",
      "I am passionate about improving customer care operations.",
      "Adapt to new technologies with ease.",
      "Value the importance of clear and open communication.",
      "Ambitious about reaching new career milestones.",
      "I can efficiently escalate issues when necessary.",
      "Emphasize the importance of customer-centric service.",
      "I am always looking to enhance my technical knowledge.",
      "Thrive in environments that promote personal growth.",
      "I have participated in several successful sales campaigns.",
      "Team-oriented and driven by achieving common goals.",
      "I am dedicated to providing top-notch technical support.",
      "Well-versed in troubleshooting and resolving technical problems.",
      "Genuinely enjoy finding solutions to customer challenges.",
      "I can quickly adapt to different customer service scenarios.",
      "Prioritize maintaining a high level of customer satisfaction.",
      "I am focused on achieving both professional and personal growth.",
      "Strong background in managing customer relationships.",
      "I have a talent for simplifying complex technical details.",
      "Strive to create a positive customer experience.",
      "Plan to develop further skills in sales management.",
      "Quick at mastering new tools and technologies.",
      "I'm known for my attention to detail in customer interactions.",
      "Eager to work in a culturally diverse setting.",
      "Aim to consistently exceed performance targets.",
      "I am proficient in using various chat communication tools.",
      "Have a strong sense of accountability and responsibility.",
      "Look forward to taking on leadership roles.",
      "I am skilled at managing time and tasks effectively.",
      "Believe in maintaining professionalism in all communications.",
      "Motivated by challenging professional goals.",
      "I can deliver comprehensive training to new hires.",
      "Dedicated to providing outstanding customer service.",
      "Focused on achieving excellence in technical support.",
      "Proud of my ability to remain calm under pressure.",
      "Aspirations include becoming a senior technical advisor.",
      "Foster a collaborative and supportive work environment.",
      "Adapt well to new processes and workflows.",
      "Believe in the importance of continuous improvement.",
      "I can provide insightful feedback to improve service.",
      "Take the initiative to solve customer issues proactively.",
      "I have mastered various evaluation techniques."
      "Committed to transparency and integrity in all dealings.",
      "Enjoy the challenge of meeting sales targets consistently.",
      "Emphasize customer satisfaction in all interactions.",
      "I have strong analytical skills to assess customer needs.",
      "Love working in a dynamic and fast-paced environment.",
      "Focused on climbing the career ladder step by step.",
      "Proficiency in multiple technical support platforms.",
      "Make customer satisfaction a top priority.",
      "Plan to leverage my skills to contribute to company growth.",
      "Quick to learn and apply new customer service strategies.",
      "Believe in creating a positive work environment.",
      "Aim to build a long-term career in customer service.",
      "Commit to continually improving my technical skills.",
      "Strive for excellence in all customer service roles.",
      "Known for my effective problem-solving abilities.",
      "Keen to take on more responsibilities over time.",
      "Highly skilled in handling high-pressure situations.",
      "I am enthusiastic about fostering team collaboration.",
      "Pursue ongoing education and professional development.",
      "Recognized for my exceptional customer service skills.",
      "Set meaningful career goals and work hard to achieve them.",
      "Enjoy providing technical support through online chat.",
      "Have a natural aptitude for understanding software systems.",
      "Believe in bridging the gap between technology and customers.",
      "Set clear and achievable career goals for myself.",
      "Responsive and effective in managing customer inquiries.",
      "Aspire to lead and mentor new customer service representatives.",
      "Pride myself on delivering high-quality service.",
      "I can quickly identify customer pain points and address them.",
      "Value effective communication and problem-solving.",
      "Look forward to contributing to a forward-thinking organization.",
      "Proven track record in improving customer satisfaction rates.",
      "I enjoy collaborating with diverse teams to achieve goals.",
      "Believe in using data to drive decision-making processes.",
      "Aim to continuously enhance my customer support skills.",
      "Known for my resilience and perseverance.",
      "Proficient in various client management tools.",
      "I can provide detailed analysis and feedback to improve processes.",
      "Dedicated to achieving a high standard of customer service.",
      "Highly adaptable to new and evolving technologies.",
      "I believe in setting high standards for myself and achieving them.",
      "Skilled in building rapport with customers quickly.",
      "I am motivated by both personal and professional growth.",
      "Enjoy tackling complex issues and finding practical solutions.",
      "Committed to fostering a positive customer experience.",
      "I can streamline workflows for better efficiency.",
      "Have a strong focus on customer satisfaction.",
      "Prioritize continuous learning and improvement.",
      "I am experienced in managing customer expectations.",
      "I can anticipate customer needs and act accordingly.",
      "Aspire to move into higher-level customer service positions.",
      "Believe in delivering value through excellent service.",
      "Strive to make every customer interaction positive.",
      "I can manage multiple tasks efficiently without compromising quality.",
      "Focus on enhancing customer experience through innovation.",
      "Look forward to being part of a dynamic team.",
      "I can identify gaps in service and work to close them.",
      "Recognize the importance of empathy in customer service.",
      "I've mastered the art of multitasking in a busy environment.",
      "Consistently aim to exceed customer expectations.",
      "Proud of my ability to foster lasting customer relationships.",
      "Believe in taking a proactive approach to problem-solving.",
      "I am adept at using various customer service platforms.",
      "Motivated by a desire to help others and facilitate solutions.",
      "I can provide excellent service under tight deadlines.",
      "Aspire to make a meaningful impact in my field.",
      "Known for my effective and clear communication skills.",
      "I have a proven ability to drive customer satisfaction.",
      "Focus on achieving both short and long-term career goals.",
      "Enjoy staying updated with the latest industry trends.",
      "Determined to reach my career aspirations through persistence.",
      "Excel at adapting to new challenges and responsibilities.",
      "I am passionate about delivering superior service.",
      "Believe in leading by example and inspiring others.",
      "I can handle complex customer queries with ease.",
      "Commit to enhancing my skills through continuous learning.",
      "Look forward to achieving top performance in my role.",
      "I can effectively manage high-stress situations.",
      "Known for my strong problem-solving capabilities.",
      "Prioritize developing strong relationships with clients.",
      "Independent and collaborative worker with a strong drive.",
      "I am skilled at identifying and resolving issues promptly.",
      "Focused on achieving high levels of customer satisfaction.",
      "Believe in fostering a positive and inclusive workplace.",
      "I am eager to expand my knowledge and expertise.",
      "Committed to excellence in customer service delivery.",
      "Ambitious about advancing my career in customer service.",
      "Excel at building rapport and engaging with customers.",
      "I am confident in handling diverse customer scenarios.",
      "Strive to make each customer interaction valuable.",
      "I can handle a high volume of customer inquiries effectively.",
      "Strength lies in understanding and meeting customer needs.",
      "Look forward to contributing to a high-performing team.",
      "I am passionate about driving customer-first initiatives.",
      "Believe in going the extra mile to solve customer problems.",
      "Adept at using data to improve customer service outcomes.",
      "Aim to become a subject-matter expert in customer relations.",
      "Focus on delivering consistent and reliable service.",
      "Known for my dedication and hard work.",
      "I can maintain composure in challenging situations.",
      "Passionate about delivering solutions that satisfy customers.",
      "I can manage and prioritize multiple customer requests.",
      "Seek to continuously elevate the quality of service delivered.",
      "Driven by the objective of making customers happy.",
      "I am committed to achieving excellence in my career.",
      "Value the importance of a supportive team environment.",
      "I can effectively communicate with technical and non-technical clients.",
      "Looking forward to leadership opportunities.",
      "I prioritize listening to customers and understanding their perspective.",
      "Believe that a customer-centered approach is essential.",
      "Focused on excelling as a technical support specialist.",
      "Strive to create a positive impact through my work.",
      "I am dedicated to personal and professional growth.",
      "Enjoy taking on new challenges and responsibilities.",
      "I am effective in managing customer expectations.",
      "Plan to leverage my skills to drive company success.",
      "Committed to upholding high standards of integrity.",
      "Believe in continuous improvement and innovation.",
      "Known for my ability to resolve issues swiftly.",
      "I aim to build strong and lasting relationships with clients.",
      "Look forward to expanding my skill set.",
      "Consider effective communication a cornerstone of success.",
      "Focus on helping the team achieve common goals.",
      "I am adaptable to changing customer needs.",
      "Motivated to provide exceptional technical support.",
      "I can deliver high levels of service with efficiency.",
      "Believe in maintaining a proactive approach to service.",
      "Aspire to be a key player in my organization's success.",
      "Value clear and consistent communication.",
      "I have a strong commitment to delivering quality service.",
      "Enjoy finding effective solutions to customer issues.",
      "I can successfully manage customer relationships.",
      "I am focused on setting and reaching career objectives.",
      "Believe in creating value through exceptional service.",
      "I am known for being a dependable team member.",
      "Commit to delivering superior service experiences.",
      "I can manage multiple priorities and deadlines.",
      "Strive to continuously improve my performance.",
      "Focused on achieving and exceeding goals.",
      "Consider empathetic communication essential for success.",
      "I am motivated to make constructive contributions.",
      "Aspire to continually develop as a customer service professional.",
      "Value teamwork and collective success.",
      "Believe in the power of positive and effective interactions.",
      "Seek to understand and fulfill customer requirements.",
      "I am proactive in identifying improvement opportunities.",
      "Focused on delivering high standards of service.",
      "I can build trust and rapport with customers.",
      "Highly motivated to achieve customer satisfaction.",
      "Have a strong reputation for providing reliable service.",
      "I strive to create positive and impactful experiences.",
      "Believe in the importance of professional integrity.",
      "Committed to my personal and team’s growth.",
      "I am skilled at managing and resolving conflicts.",
      "Value being part of an innovative and dynamic team.",
      "Dedicated to meeting and exceeding performance metrics.",
      "I can deliver effective technical support remotely.",
      "Aspire to take on challenging roles and excel.",
      "Commit to maintaining professionalism in all interactions.",
      "Recognize the value of constructive feedback.",
      "Look forward to new learning opportunities.",
      "Focused on making a difference through my work.",
      "I am proactive in anticipating and solving problems.",
      "Strength lies in building and maintaining strong client relationships.",
      "I can provide exceptional service in a high-volume setting.",
      "I am motivated by the opportunity to improve processes.",
      "Committed to ensuring customer satisfaction.",
      "Aspire to continuously enhance my skill set.",
      "Believe in delivering beyond customer expectations.",
      "I can leverage data to drive decision-making.",
      "Value clear and proactive communication.",
      "I am known for consistent and reliable performance.",
      "Embrace change and growth in my career.",
      "Strive to exceed customer service standards.",
      "Motivated to provide solutions that meet client needs.",
      "Focused on achieving success through collaboration.",
      "I am driven by a passion for customer service excellence.",
      "Eager to contribute to a team-oriented environment.",
      "I can handle high-volume inquiries with ease.",
      "Believe in continuous personal and professional development.",
      "Committed to setting and achieving high standards.",
      "Focused on advancing my career through hard work.",
      "Can quickly adapt to diverse and dynamic work environments.",
      "Passionate about delivering exceptional service.",
      "I strive to align my goals with company objectives.",
      "I am dedicated to ensuring positive customer experiences.",
      "Aspire to be a leader in the customer service industry.",
      "Committed to upholding the values of integrity and excellence.",
      "I can identify and implement effective solutions promptly.",
      "Focus on building a strong foundation for career advancement.",
      "I can quickly resolve technical issues to customer satisfaction.",
      "Believe in the importance of empathy in customer interactions.",
      "Pride myself on delivering effective and efficient solutions.",
      "I am dedicated to continuous professional growth.",
      "Motivated to create value for my team and organization.",
      "Committed to delivering outstanding customer service.",
      "I can proficiently manage varied customer inquiries.",
      "Seek to enhance my career through ongoing learning.",
      "Dedicated to providing high-quality service consistently.",
      "Focused on achieving both individual and team goals.",
      "I can handle complex customer service scenarios skillfully.",
      "Value transparency and effective communication.",
      "I am driven by a commitment to excellence.",

    ]
}


continue_yes = {
    "responses": [
	    "Sounds good! Let's move forward.",
	    "I'm happy to help. Please proceed.",
	    "That's correct! Let's continue 💬",
	    "Got it! Next steps are...",
	    "Affirmative, let's go ahead.",
	    "Excellent, what's the next question?",
	    "That's okay, let's try again.",
	    "I completely agree. What's next?",
	    "That's correct! 👍",
	    "I'm on the same page. Please proceed.",
	    "That's awesome! What's the next step?",
	    "I'm excited to see what's next!",
	    "That sounds like a plan. Let's do it.",
	    "I'm with you on that. What's next?",
	    "That's a great idea! Let's run with it.",
	    "That's a good point. Let's build on that.",
	    "I'm good with that. What's the next move?",
	    "That's the right approach. Let's continue.",
	    "That's a great start! What's next?",
	    "I'm on board with that. Let's proceed.",
	    "That's a fantastic idea! Let's make it happen.",
	    "Oki",
	    "Okay",
	    "Ok",
        "Yes, I would like to continue.",
        "Sure, I'm in!",
        "Absolutely, let's go ahead.",
        "Count me in!",
        "I agree, let's proceed.",
        "I'm on board with this.",
        "Yep, sounds good to me.",
        "Definitely, let's do this.",
        "Okay, I'm ready to move forward.",
        "Alright, let's continue.",
        "Of course, I'm in favor.",
        "👍 I'm in.",
        "Yes, please proceed.",
        "I'm all set to continue.",
        "For sure, let's keep going.",
        "I'm okay with that, let's proceed.",
        "Certainly, I agree.",
        "👌 Let's go for it.",
        "Fine by me, let's move forward.",
        "I'm down for this!",
        "Sounds good, I'm in!",
        "Yes, I agree!",
        "I'm on board!",
        "Works for me!",
        "I accept!",
        "Alright, let's do it!",
        "I'm down!",
        "Sounds like a plan!",
        "Okay, I'm in!",
        "Yeah, I'm good with that!",
        "I'm cool with it!",
        "I agree, let's move forward!",
        "I'm on the same page!",
        "Sounds good to me!",
        "Okay, I accept!",
        "That works for me! 👍",
        "I'm in agreement!",
        "I concur!",
        "I'm down for that!",
        "I'm good with that!",
        "I'm okay with that!",
        "Yeah, that works!",
        "I'm in!",
        "Alright, sounds good!",
        "I'm on board with that!",
        "I'm good, let's proceed!",
        "Sounds legit!",
        "I'm cool with it, thanks!",
        "That sounds good to me!",
        "I accept the offer!",
        "I'm in, thanks!",
        "Yeah, that sounds good!",
        "I'm down with that!",
        "I'm okay with that, thanks!",
        "Alright, I'm in!",
        "Sounds good, I'm good!",
        "I'm good to go!",
        "I'm in agreement, thanks!",
        "I'm down for it!",
        "Yeah, I'm in!",
        "Sounds good, let's do it!",
        "I'm okay with that, yeah!",
        "I'm in, yeah!",
        "That sounds good, I'm in!",
        "I'm good with that, thanks!",
        "Sure, let's keep going.",
            "Yes, I'd like to proceed.",
            "I'm ready to continue.",
            "Absolutely, let's move forward.",
            "Yes, please continue.",
            "Let's do this!",
            "I agree, let's carry on.",
            "Sure thing, let's proceed.",
            "Sounds good, let's move on.",
            "Yes, I'm in.",
            "Okay, let's go ahead.",
            "I'm good to go.",
            "Yup, let's continue.",
            "All set to proceed.",
            "Definitely, let's keep moving.",
            "Alright, let's do it.",
            "I'm on board, let's go.",
            "Yes, let's proceed with this.",
            "Happy to continue.",
            "Understood, I'm ready to move forward.",
            "Fine by me, let's go ahead.",
            "Let’s get rolling!",
            "I'm all for it, proceed.",
            "Yep, let's go on.",
            "Cool, let's keep moving.",
            "Sure, I'm for it.",
            "Proceed as planned.",
            "Yes, let's carry forth.",
            "I support this, go ahead.",
            "Agreed, let's continue.",
            "Count me in, go ahead.",
            "Onward we go!",
            "Yes, I consent to continue.",
            "👌 Let's continue.",
            "I’m agreeable to that.",
            "Confirmed, let's move ahead.",
            "Totally, let's move on.",
            "Amen to that, let's go.",
            "I'm down, proceed.",
            "With pleasure, let's proceed.",
            "Yes, that's fine with me.",
            "I'm game, let's move on.",
            "I'm good with that, let's carry on.",
            "Alrighty, let's continue.",
            "Yes, let's advance.",
            "Let's keep the momentum going.",
            "Cool with me, proceed.",
            "Happy to move forward.",
            "You bet, let’s keep going.",
            "Affirmative, proceed.",
            "I'm in agreement, continue.",
            "Okay, let's advance.",
            "Of course, carry on.",
            "Let's get on with it.",
            "Yes, I'm good with that.",
            "Of course, moving on.",
            "Roger that, let's continue.",
            "Sure, let’s get going.",
            "All right, go ahead.",
            "Solid, let’s proceed.",
            "Let's proceed, I'm ready.",
            "We can continue.",
            "Affirmative, let's advance.",
            "I'm okay with moving forward.",
            "Proceed, I'm ready.",
            "Yeppers, let’s move on.",
            "I'm for continuing.",
            "Sure, let’s move ahead.",
            "Thumbs up, let's proceed!",
            "Go ahead, I consent.",
            "Surely, let's move forward.",
            "You got it, let's continue.",
            "Affirmative, let’s carry on.",
            "Let’s go forward.",
            "Absolutely, let’s move on.",
            "I accept, continue.",
            "Let's keep on.",
            "I approve, proceed.",
            "So be it, let’s continue.",
            "True that, proceed.",
            "You have my consent, move on.",
            "Let's not stop.",
            "I’m up for it, let’s go.",
            "Fine, keep going.",
            "Proceeding is agreeable.",
            "Exactly, let’s continue.",
            "Yes, that's a go.",
            "Great, let’s go ahead.",
            "Let's keep up the pace.",
            "I concur, let's continue.",
            "Sure, move ahead.",
            "That's a yes, let's proceed.",
            "Absolutely, continue.",
            "You betcha, let's keep moving.",
            "I'm clear, proceed.",
            "Right on, keep moving.",
            "Yup, go ahead.",
            "Fine with me, move on.",
            "By all means, continue.",
            "That's fine, let's move ahead.",
            "Without a doubt, proceed.",
            "I’m in, let’s continue.",
            "I give the go-ahead.",
            "Sure thing, move on.",
            "Yes, let’s carry forward.",
            "Do carry on.",
            "I’m with you, let's move on.",
            "Let’s dive right in!",
            "Right, continue.",
            "That works, let's proceed.",
            "Yep, let's keep this going.",
            "Let's keep pushing forward.",
            "Permission granted, go ahead.",
            "I validate, proceed.",
            "Okay, I’m on board.",
            "Certainly, let’s move forward.",
            "With all my heart, continue.",
            "Alright, move along.",
            "I am prepared, let’s continue.",
            "Go ahead, thumbs up.",
            "Yes, this works, continue.",
            "Let’s hit it!",
            "You may proceed.",
            "I’ll say yes, continue.",
            "I accept, carry on.",
            "Keep going, I’m ready.",
            "Confirmed, let's move forward.",
            "Let's continue this journey.",
            "I validate the process, proceed.",
            "All good, let's move forward.",
            "That’s cool, continue.",
            "Totally on board, let’s go!",
            "Let's maintain this flow.",
            "Rock on, let's keep moving.",
            "That's fine by me, proceed.",
            "I acknowledge, let's continue.",
            "Persuaded, go ahead.",
            "Well said, keep going.",
            "Let's keep the ball rolling.",
            "Right, keep moving.",
            "Yes, end of story, continue.",
            "Solid choice, let's go.",
            "I’m compliant, continue.",
            "Itinerary approved, proceed.",
            "Okie dokie, let’s go!",
            "Right then, move forward.",
            "Let’s keep trekking.",
            "Let’s remain steadfast.",
            "Let’s persist.",
            "Certainly, let’s advance.",
            "Undoubtedly, proceed.",
            "Count me in, let’s keep moving.",
            "Clear as day, continue.",
            "No objections here, proceed.",
            "Let’s journey onwards.",
            "Steadfast, let’s go!",
            "All right, let’s press on.",
            "I approve, continue.",
            "Agreed upon, proceed.",
            "I commit, let's move forward.",
            "Proceed, with my blessings.",
            "Understood, let's move ahead.",
            "Agree, continue.",
            "I am positive, continue.",
            "Absolutely, let’s proceed.",
            "No qualms, move on.",
            "Yes, this is okay, proceed.",
            "I’m good with that, carry on.",
            "Spot on, keep going.",
            "I’m prepared, continue.",
            "Got it, let's proceed.",
            "All right, let's advance.",
            "Yessir, let’s go ahead.",
            "Fine, let’s progress.",
            "Affirmative, let’s move forward.",
            "Let's move on, sure.",
            "Conclude and continue.",
            "I give my nod, go ahead.",
            "I say yes, move on.",
            "Confirmed, carry forth."
    ]
}


thanks = {
    "responses": [
      "Thank you!",
      "Much appreciated!",
      "Thanks a bunch!",
      "I really appreciate it!",
      "Thanks for your help!",
      "Thank you so much!",
      "I'm grateful for your assistance.",
      "Thanks a lot!",
      "Cheers!",
      "Thanks!",
      "Thanks a million!",
      "I'm forever grateful!",
      "Many thanks!",
      "Thanks for your support!",
      "I can't thank you enough!",
      "Thanks for everything!",
      "Thank you kindly!",
      "You're the best!",
      "Thanks again!",
      "Thank you, I appreciate it a lot!",
      "Grateful for your help!",
      "Big thanks!",
      "Thank you so very much!",
      "Thumbs up!",
      "Appreciate your time!",
      "Thanks, I owe you one!",
      "You're a lifesaver!",
      "Thank you, this means a lot!",
      "Thanks, you're awesome!",
      "Gracias!",
      "Thank you from the bottom of my heart!",
      "It means the world to me, thanks!",
      "Appreciate it!",
      "Thanks for the effort!",
      "Thanks a ton!",
      "Thanks in advance!",
      "Thank you for your kindness!",
      "Thank you for your cooperation!",
      "I'm grateful to you.",
      "Thank you for your patience!",
      "Much obliged!",
      "Your help was invaluable.",
      "I'm so thankful!",
      "Couldn't have done it without you!",
      "You've been a big help, thanks!",
      "Thank you for your understanding.",
      "I appreciate your assistance.",
      "I'd like to thank you sincerely.",
      "Thanks for the input!",
      "Thanks for being there!",
      "Thank you for your dedication!",
      "Thank you for the information!",
      "Thanks, that's very kind of you!",
      "Thankful for your guidance.",
      "Thanks for your quick response!",
      "Thank you for the update!",
      "You really helped me out, thanks!",
      "Thanks for your quick action!",
      "You have my gratitude.",
      "Thanks for being so helpful!",
      "Hats off to you!",
      "Thanks for the advice!",
      "Thank you for your service!",
      "You've been very supportive, thanks!",
      "Bless you!",
      "I appreciate your prompt attention.",
      "Thanks for your consideration.",
      "Thank you for your flexibility!",
      "Really appreciate your input.",
      "A big thanks to you!",
      "Thank you, this is great!",
      "Thanks, you're a star!",
      "I'm very grateful.",
      "Thanks for your suggestion!",
      "Thank you, that's exactly what I needed!",
      "Appreciate your expertise!",
      "Thanks for listening!",
      "Kudos!",
      "Thank you for your thoroughness.",
      "Thank you, that helps a lot!",
      "Thanks, I really needed that!",
      "Thanks for the heads up!",
      "Thanks for your prompt reply!",
      "Truly appreciate your feedback.",
      "Thanks for the clarification!",
      "You're a gem, thanks!",
      "Thank you for your time and effort.",
      "Thanks for taking care of that.",
      "Greatly appreciate your help.",
      "Thank you for your attention.",
      "Thanks for understanding!",
      "I'm indebted to you.",
      "Wonderful job, thanks!",
      "Thanks for your diligence.",
      "Thanks for resolving this quickly.",
      "Thank you, I couldn't have done it without you.",
      "A huge thank you!",
      "Thank you for your promptness.",
      "You've been incredibly helpful, thanks!",
      "Thank you so much for your support!",
      "Your help is greatly valued.",
      "Sincerely, thanks!",
      "Thank you for your assistance.",
      "Thanks for making this easy!",
      "Deeply grateful for your help.",
      "I appreciate your commitment!",
      "Big thumb up!",
      "Thanks for your patience and expertise.",
      "Appreciate your understanding.",
      "Thank you, it means a lot to me.",
      "Thank you for this opportunity.",
      "Thank you for reaching out!",
      "Your support has been invaluable.",
      "Thank you for your clarification.",
      "I'm very thankful!",
      "Thanks for having my back.",
      "Thanks for the heads-up.",
      "I'm sincerely grateful.",
      "Thanks for your consideration!",
      "Your help has been so important.",
      "I appreciate your quick response.",
      "Thank you very much!",
      "Thank you, you've been a great help!",
      "I'm thankful for your input.",
      "Thanks a heap!",
      "Thank you, it's been a great help.",
      "Thank you for taking the time!",
      "Really appreciate your efforts.",
      "Thanks a lot, I really appreciate it.",
      "Your advice was very helpful, thanks!",
      "Thank you, I value your input.",
      "I really appreciate your time.",
      "Thanks for your patience!",
      "Thanks for the quick follow-up!",
      "Thanks, much appreciated.",
      "Appreciate your cooperation!",
      "Thanks, got it!",
      "Thank you for your quick action.",
      "Many thanks for your understanding!",
      "Thank you for solving the issue.",
      "Thanks for assisting me.",
      "Thank you for your excellent service.",
      "Thanks for the excellent job!",
      "I'm grateful for your advice.",
      "Thank you for being understanding!",
      "Your assistance is much appreciated.",
      "Thanks, I appreciate your support.",
      "Thank you for the prompt reply.",
      "Thanks for your trust.",
      "Appreciate your immediate action!",
      "Thanks for contributing!",
      "Many thanks for your quick response!",
      "Thanks for the feedback!",
      "Grateful for your quick turnaround.",
      "Your quick help is appreciated.",
      "Thanks for your swift help!",
      "Countless thanks!",
      "Thanks for confirming!",
      "Thank you, you've been a great help.",
      "Appreciate your quick action.",
      "Thank you for the prompt service.",
      "Thank you for going the extra mile!",
      "Thanks for the invaluable help.",
      "Thank you for your insights.",
      "I appreciate your quick help!",
      "Thanks for the clarity.",
      "Many thanks for your cooperation.",
      "Thanks a zillion!",
      "Thank you for your kind words.",
      "Thanks for facilitating this.",
      "Thank you for your kind assistance!",
      "Thanks for the excellent support.",
      "Appreciate your timely help!",
      "Thanks for the detailed response!",
      "Thank you, I’m truly grateful.",
      "Thanks for stepping up!",
      "Thanks for covering this.",
      "Appreciate your transparency.",
      "Thanks for the great suggestion!",
      "Thanks for handling this so well.",
      "Super thankful!",
      "Thanks for the message.",
      "Thank you for your troubleshooting!",
      "Thank you so very much for that.",
      "Thanks for fixing this!",
      "Thanks for the precise instructions.",
      "Thank you for your dependability.",
      "Much gratitude!",
      "Thank you, that's greatly helpful.",
      "I highly value your help.",
      "Thanks, that's very helpful!",
      "Thank you for your promptness!",
      "Thanks for alleviating the problem.",
      "I greatly appreciate your support.",
      "Thanks for attending to this.",
      "I’m thankful for your guidance.",
      "It's much appreciated, thanks!",
      "Thank you for addressing this.",
      "Much gratitude for your help.",
      "Thank you for your commitment!",
      "Thank you, that's a big relief.",
      "Thanks, that's perfect.",
      "Thank you for the comprehensive answer.",
      "Appreciate your input!",
      "Thanks so much for your swift action.",
      "Thanks for your cooperation!",
      "Appreciate your time and effort.",
      "Thank you for always being reliable.",
      "Thank you for your attentiveness!",
      "Thanks, your assistance is much appreciated.",
      "Appreciate the clarity!",
      "Thanks, your help is greatly valued.",
      "Thank you for addressing this quickly.",
      "Sincere thanks!",
      "Thanks, you're a great help.",
      "Thanks, you've been very effective!",
      "Thank you for your perseverance.",
      "I appreciate your rapid help.",
      "Thank you, your help means a lot.",
      "Your prompt reply is much appreciated.",
      "Thanks, truly appreciate it.",
      "Big thanks for your help!",
      "Thanks for dealing with that so quickly.",
      "Thank you, that was very efficient.",
      "Appreciate your prompt support!",
      "Thanks, you've been a big help.",
      "Thank you for the useful insights.",
      "I treasure your advice, thanks!",
      "Thanks for aiding me.",
      "Thanks so much for the speedy reply.",
      "I'm grateful for your prompt service.",
      "Thank you for the helpful suggestion.",
      "Thank you for the thorough explanation!",
      "Thank you again for all your efforts.",
      "Thanks for your prompt reaction!",
      "Thanks for being so understanding.",
      "Thanks for assisting in this quickly.",
      "Thanks, I really value your input.",
      "Highly appreciate your response.",
      "Thanks for giving clarity on this.",
      "Thank you for addressing the issue.",
      "Your swift attention is much appreciated.",
      "Thanks for helping to sort this!",
      "Thank you, your assistance is invaluable.",
      "Thanks, your expertise was very helpful.",
      "Thank you, I appreciate your support.",
      "Thanks, this makes things easier.",
      "Thank you for being available!",
      "Grateful for your prompt action.",
      "Thank you, I'm grateful for your time.",
      "Thanks so much for acting quickly.",
      "Thanks, it was very kind of you.",
      "Thank you for stepping in!",
      "Appreciate your thorough help.",
      "Thanks, your response is appreciated.",
      "Thank you for the detailed feedback!",
      "Thanks for taking prompt action.",
      "Grateful for your cooperation!",
      "Thanks for your swift input.",
      "Thank you, I appreciate the advice.",
      "Thanks for sorting this promptly.",
      "Thank you, that was quick!",
      "I appreciate your assistance, thank you.",
      "Thank you for the flawless execution.",
      "Thank you for keeping me updated!",
      "Truly appreciate your detailed explanation.",
      "Thanks for providing this so swiftly!"
    ]
  }



def merge_examples(sample_type: str, sample: dict):
    return [dspy.Example({"utterance": f, "utterance_type": sample_type}).with_inputs("utterance") for f in list(sample.values())[0]]

random.shuffle(greetings['hello_messages'])
greetings['hello_messages'] = greetings['hello_messages'][:300]


examples: List[dspy.Example] = []
all_types = {
    "greetings": greetings,
    "company": company, 
    "feedbacks": feedbacks, 
    "continue_later": continue_later, 
    "stop_continue": stop_continue,
    "trouble": trouble,
    "himself": himself,
    "continue_yes": continue_yes,
    "thanks": thanks,
}

for sample_type, sample in all_types.items():
    examples.extend(merge_examples(sample_type, sample))
    print(sample_type, len(list(sample.values())[0]))

h_data = []
for ex in examples:
    h_data.append(
        {"text": ex['utterance'], "label": ex['utterance_type']}
    )

examples_dataset = Dataset.from_list(h_data)

print(len(examples))



greetings 300

company 269

feedbacks 237

continue_later 236

stop_continue 218

trouble 229

himself 271

continue_yes 269

thanks 267

2296

In [11]:
import pickle

with open("examples_v2.pkl", "wb") as file:
    pickle.dump(examples, file)

In [4]:
import pickle

# with open("examples_v2.pkl", "rb") as file:
#     pickle.dump(examples, file)

examples = None
with open("examples_v2.pkl", "rb") as file:
    examples = pickle.load(file)

In [26]:
class_names = list(all_types.keys())

label2id = {label: i for i, label in enumerate(class_names)}

h_data = []
for ex in examples:
    h_data.append(
        {"text": ex['utterance'], "label": label2id[ex['utterance_type']]}
    )
examples_dataset = Dataset.from_list(h_data)

train_test_split = examples_dataset.train_test_split(test_size=0.6, seed=42)

train_dataset = train_test_split['train']
__dataset = train_test_split['test'].train_test_split(test_size=0.3, seed=42)
test_dataset = __dataset['train']
val_dataset = __dataset['test']
del __dataset

train_dataset, test_dataset, val_dataset


(Dataset({
     features: ['text', 'label'],
     num_rows: 918
 }),
 Dataset({
     features: ['text', 'label'],
     num_rows: 964
 }),
 Dataset({
     features: ['text', 'label'],
     num_rows: 414
 }))

In [15]:
import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
from huggingface_hub import HfFolder, notebook_login

In [33]:
%%capture
%pip install ipywidgets tensorboard

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:

model_id = "roberta-base"
repository_id = "FacebookAI/roberta-base"


# Preprocessing
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

# This function tokenizes the input text using the RoBERTa tokenizer. 
# It applies padding and truncation to ensure that all sequences have the same length (256 tokens).
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=256)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))


# Set dataset format
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])




Map:   0%|          | 0/918 [00:00<?, ? examples/s]

Map:   0%|          | 0/414 [00:00<?, ? examples/s]

Map:   0%|          | 0/964 [00:00<?, ? examples/s]

In [17]:
# We will need this to directly output the class names when using the pipeline without mapping the labels later.
# Extract the number of classes and their names
class_names = list(all_types.keys())

# Create an id2label mapping
id2label = {i: label for i, label in enumerate(class_names)}
num_labels = len(class_names)
print(f"number of labels: {num_labels}")
print(f"the labels: {class_names}")



# Update the model's configuration with the id2label mapping
config = AutoConfig.from_pretrained(model_id)
config.update({"id2label": id2label})



number of labels: 9

the labels: ['greetings', 'company', 'feedbacks', 'continue_later', 'stop_continue', 'trouble', 'himself', 
'continue_yes', 'thanks']

/Users/beltre.wilton/miniforge3/envs/tars_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [19]:
# Model
model = RobertaForSequenceClassification.from_pretrained(model_id, config=config)
model.to(device)

# TrainingArguments
training_args = TrainingArguments(
    output_dir=repository_id,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard",
    push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/beltre.wilton/miniforge3/envs/tars_env/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
# Fine-tune the model
trainer.train()

  0%|          | 0/575 [00:00<?, ?it/s]

{'loss': 2.2085, 'grad_norm': 3.6473913192749023, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.09}
{'loss': 2.1985, 'grad_norm': 5.287281513214111, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.17}
{'loss': 2.1871, 'grad_norm': 4.457653045654297, 'learning_rate': 3e-06, 'epoch': 0.26}
{'loss': 2.1863, 'grad_norm': 3.334641933441162, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.35}
{'loss': 2.1707, 'grad_norm': 4.754641056060791, 'learning_rate': 5e-06, 'epoch': 0.43}
{'loss': 2.183, 'grad_norm': 8.909841537475586, 'learning_rate': 6e-06, 'epoch': 0.52}
{'loss': 2.1645, 'grad_norm': 3.7907962799072266, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.61}
{'loss': 2.1249, 'grad_norm': 4.324082851409912, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.7}
{'loss': 2.0467, 'grad_norm': 5.227200508117676, 'learning_rate': 9e-06, 'epoch': 0.78}
{'loss': 1.9336, 'grad_norm': 10.993483543395996, 'learning_rate': 1e-05, 'epoch': 0.87}
{'loss': 1.777, 'grad_norm': 11.76338

  0%|          | 0/52 [00:00<?, ?it/s]

{'eval_loss': 1.3622454404830933, 'eval_runtime': 1.7359, 'eval_samples_per_second': 238.488, 'eval_steps_per_second': 29.955, 'epoch': 1.0}
{'loss': 1.4599, 'grad_norm': 16.374794006347656, 'learning_rate': 1.2e-05, 'epoch': 1.04}
{'loss': 1.2248, 'grad_norm': 11.452479362487793, 'learning_rate': 1.3000000000000001e-05, 'epoch': 1.13}
{'loss': 0.9186, 'grad_norm': 11.178313255310059, 'learning_rate': 1.4000000000000001e-05, 'epoch': 1.22}
{'loss': 0.6969, 'grad_norm': 12.407782554626465, 'learning_rate': 1.5e-05, 'epoch': 1.3}
{'loss': 0.5866, 'grad_norm': 21.539356231689453, 'learning_rate': 1.6000000000000003e-05, 'epoch': 1.39}
{'loss': 0.3763, 'grad_norm': 5.552234172821045, 'learning_rate': 1.7000000000000003e-05, 'epoch': 1.48}
{'loss': 0.3141, 'grad_norm': 18.024282455444336, 'learning_rate': 1.8e-05, 'epoch': 1.57}
{'loss': 0.2091, 'grad_norm': 21.27614402770996, 'learning_rate': 1.9e-05, 'epoch': 1.65}
{'loss': 0.1733, 'grad_norm': 1.7559014558792114, 'learning_rate': 2e-05, 

  0%|          | 0/52 [00:00<?, ?it/s]

...

  0%|          | 0/52 [00:00<?, ?it/s]

{'eval_loss': 0.04188018664717674, 'eval_runtime': 1.4771, 'eval_samples_per_second': 280.284, 'eval_steps_per_second': 35.205, 'epoch': 4.0}
{'loss': 0.0653, 'grad_norm': 33.6285400390625, 'learning_rate': 4.7e-05, 'epoch': 4.09}
{'loss': 0.0065, 'grad_norm': 0.024330681189894676, 'learning_rate': 4.8e-05, 'epoch': 4.17}
{'loss': 0.0031, 'grad_norm': 2.910928964614868, 'learning_rate': 4.9e-05, 'epoch': 4.26}
{'loss': 0.0329, 'grad_norm': 0.025515111163258553, 'learning_rate': 5e-05, 'epoch': 4.35}
{'loss': 0.002, 'grad_norm': 0.03984646871685982, 'learning_rate': 4.3333333333333334e-05, 'epoch': 4.43}
{'loss': 0.0018, 'grad_norm': 0.0261151771992445, 'learning_rate': 3.6666666666666666e-05, 'epoch': 4.52}
{'loss': 0.0018, 'grad_norm': 0.022934338077902794, 'learning_rate': 3e-05, 'epoch': 4.61}
{'loss': 0.1028, 'grad_norm': 0.02120666764676571, 'learning_rate': 2.3333333333333336e-05, 'epoch': 4.7}
{'loss': 0.0016, 'grad_norm': 0.015421452932059765, 'learning_rate': 1.666666666666666

  0%|          | 0/52 [00:00<?, ?it/s]

{'eval_loss': 0.03153407946228981, 'eval_runtime': 1.4697, 'eval_samples_per_second': 281.698, 'eval_steps_per_second': 35.382, 'epoch': 5.0}
{'train_runtime': 154.1793, 'train_samples_per_second': 29.771, 'train_steps_per_second': 3.729, 'train_loss': 0.5380906649144447, 'epoch': 5.0}


TrainOutput(global_step=575, training_loss=0.5380906649144447, metrics={'train_runtime': 154.1793, 'train_samples_per_second': 29.771, 'train_steps_per_second': 3.729, 'total_flos': 63690239196540.0, 'train_loss': 0.5380906649144447, 'epoch': 5.0})

In [21]:
# Evaluate the model
trainer.evaluate()

  0%|          | 0/52 [00:00<?, ?it/s]

{'eval_loss': 0.03153407946228981,
 'eval_runtime': 1.5691,
 'eval_samples_per_second': 263.851,
 'eval_steps_per_second': 33.141,
 'epoch': 5.0}

In [22]:
import os
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments


# Define the paths where you want to save the model and tokenizer
model_save_path = "roberta_base_ft"
tokenizer_save_path = "roberta_base_ft_tokenizer"

# Create directories if they don't exist
os.makedirs(model_save_path, exist_ok=True)
os.makedirs(tokenizer_save_path, exist_ok=True)

# Save the model
model.save_pretrained(model_save_path)

# Save the tokenizer
tokenizer.save_pretrained(tokenizer_save_path)


('roberta_base_ft_tokenizer/tokenizer_config.json',
 'roberta_base_ft_tokenizer/special_tokens_map.json',
 'roberta_base_ft_tokenizer/vocab.json',
 'roberta_base_ft_tokenizer/merges.txt',
 'roberta_base_ft_tokenizer/added_tokens.json',
 'roberta_base_ft_tokenizer/tokenizer.json')

In [23]:
# Load the fine-tuned model and tokenizer
fine_tuned_model = RobertaForSequenceClassification.from_pretrained(model_save_path)
fine_tuned_tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_save_path)

In [27]:
class_names = list(all_types.keys())

label2id = {label: i for i, label in enumerate(class_names)}

h_data = []
for ex in examples:
    h_data.append(
        {"text": ex['utterance'], "label": label2id[ex['utterance_type']]}
    )
examples_dataset = Dataset.from_list(h_data)

train_test_split = examples_dataset.train_test_split(test_size=0.6, seed=42)

train_dataset = train_test_split['train']
__dataset = train_test_split['test'].train_test_split(test_size=0.3, seed=42)
test_dataset = __dataset['train']
val_dataset = __dataset['test']
del __dataset

train_dataset, test_dataset, val_dataset


(Dataset({
     features: ['text', 'label'],
     num_rows: 918
 }),
 Dataset({
     features: ['text', 'label'],
     num_rows: 964
 }),
 Dataset({
     features: ['text', 'label'],
     num_rows: 414
 }))

In [38]:
fine_tuned_model.eval()

t = test_dataset[random.randint(0, 7)]
input_text = t['text']
true_class = t['label']

# Tokenize the input text
inputs = fine_tuned_tokenizer.encode_plus(
    input_text,
    add_special_tokens=True,
    max_length=128,
    padding='max_length',
    return_tensors='pt',
    truncation=True
)

# Extract input tensors
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

# Perform inference
with torch.no_grad():
    outputs = fine_tuned_model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

# Print the result
print(f'Input Text: "{input_text}"')
print(f'Predicted Class: {id2label[predicted_class]}')
print(f'True      Class: {id2label[true_class]}')

Input Text: "Thumbs up!"

Predicted Class: thanks

True      Class: thanks

In [39]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score


all_preds = []
all_trues= []

with torch.no_grad():
    for test in tqdm(test_dataset, desc="Evaluating [test dataset]", total=len(test_dataset), ncols=110):
        text_utterance = test['text']
        true_class = test['label']
        # Tokenize the input text
        inputs = fine_tuned_tokenizer.encode_plus(
            text_utterance,
            add_special_tokens=True,
            max_length=128,
            padding='max_length',
            return_tensors='pt',
            truncation=True
        )
        # Extract input tensors
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        outputs = fine_tuned_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()

        all_preds.append(predicted_class)
        all_trues.append(true_class)
        

# Calculate accuracy
accuracy = accuracy_score(all_trues, all_preds)
print(f'Test Accuracy: {accuracy:.4f}')


Evaluating [test dataset]: 100%|████████████████████████████████████████████| 964/964 [00:58<00:00, 16.42it/s]


Test Accuracy: 0.9865

In [89]:
classifier = pipeline("zero-shot-classification", 
                      model="facebook/bart-large-mnli", 
                      device=device) 



In [102]:
import numpy as np

all_preds = []
all_trues= []
candidate_labels = class_names

for test in tqdm(test_dataset, desc="Evaluating [test dataset]", total=len(test_dataset), ncols=110):
    text_utterance = test['text']
    true_class = test['label']
    sequences = text_utterance
    results = classifier(sequences, candidate_labels) 
    mx = np.argmax(results['scores'])
    label = results['labels'][mx]
    id = label2id[label]
    
    all_preds.append(id)
    all_trues.append(true_class)
        

# Calculate accuracy
accuracy = accuracy_score(all_trues, all_preds)
print(f'Test Accuracy: {accuracy:.4f}')


Evaluating [test dataset]: 100%|████████████████████████████████████████████| 852/852 [06:36<00:00,  2.15it/s]


Test Accuracy: 0.6608

In [120]:
import psycopg2
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector

import numpy as np
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel


tokenizer_embed = AutoTokenizer.from_pretrained('bert-base-uncased')
model_embed = AutoModel.from_pretrained('nomic-ai/nomic-embed-text-v1.5', trust_remote_code=True, safe_serialization=True)
model_embed.eval()


def embedd(text: str):
    def mean_pooling(model_output, attention_mask):
        token_embeddings = model_output[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


    encoded_input = tokenizer_embed(text, padding=True, truncation=True, return_tensors='pt')

    # + matryoshka_dim = 512

    with torch.no_grad():
        model_output = model_embed(**encoded_input)

    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    # + embeddings = F.layer_norm(embeddings, normalized_shape=(embeddings.shape[1],))
    # + embeddings = embeddings[:, :matryoshka_dim]
    embeddings = F.normalize(embeddings, p=2, dim=1)

    return np.array(embeddings)[0]




def app_benchmark_topic(text: str) -> str:
    try:
        connection = psycopg2.connect(user="drfadul",
                                    password="dROG@dijoFadul",
                                    host="localhost",
                                    port="5432",
                                    database="synaia")
        register_vector(connection)
        cursor = connection.cursor()
        text = embedd(text=text)
        data = (text, text)
    
        postgreSQL_select_Query = "SELECT topic, 1 - (text_vec <=> %s::vector) AS similarity FROM app_benchmark_topic ORDER BY text_vec <=> %s::vector LIMIT 1"

        cursor.execute(postgreSQL_select_Query, data)
        mobile_records = cursor.fetchall()

        for row in mobile_records:
            # print("id = ", row[0], )
            # print("topic = ", row[1])
            # print("similarity = ", row[2])
            # return  row[1]
            return row[0], row[1]

    except (Exception, psycopg2.Error) as error:
        print("\x1b[1;31m Error while fetching data from PostgreSQL \x1b[1;31m", error)

    finally:
        # closing database connection.
        if connection:
            cursor.close()
            connection.close()
            # print("\x1b[1;30m PostgreSQL connection is closed \x1b[1;30m")


def insert_app_benchmark_topic(topic: str, text: str):
    try:
        connection = psycopg2.connect(user="drfadul",
                                    password="dROG@dijoFadul",
                                    host="localhost",
                                    port="5432",
                                    database="synaia")
        
        register_vector(connection)
        cursor = connection.cursor()
        data = [
            (topic, embedd(text=text), text)
        ]

        execute_values(cursor, "INSERT INTO app_benchmark_topic (topic, text_vec, text) VALUES %s", data)
        connection.commit()

        # print('\nSuccess!')

    except (Exception, psycopg2.Error) as error:
        print("Error while INSERTING data from PostgreSQL", error)

    finally:
        # closing database connection.
        if connection:
            cursor.close()
            connection.close()
            # print("PostgreSQL connection is closed")



<All keys matched successfully>


In [109]:
# INSERT TRAIN IN DATA BASE....
for test in tqdm(train_dataset, desc="Inserting  in[app_benchmark_topic] ...", total=len(train_dataset), ncols=110):
    text_utterance = test['text']
    true_class = test['label']
    insert_app_benchmark_topic(true_class, text_utterance)

Inserting  in[app_benchmark_topic] ...: 100%|███████████████████████████████| 811/811 [00:49<00:00, 16.35it/s]


In [127]:
all_preds = []
all_trues= []

for test in tqdm(test_dataset, desc="Testing  [app_benchmark_topic] ...", total=len(test_dataset), ncols=110):
    text_utterance = test['text']
    true_class = test['label']
    pred = app_benchmark_topic(text_utterance)
    id = int(pred[0])
    all_preds.append(id)
    all_trues.append(true_class)
        

# Calculate accuracy
accuracy = accuracy_score(all_trues, all_preds)
print(f'Test Accuracy: {accuracy:.4f}')


Testing  [app_benchmark_topic] ...: 100%|███████████████████████████████████| 852/852 [01:05<00:00, 13.11it/s]


Test Accuracy: 0.9648

In [231]:
from datetime import datetime

sending_date = datetime.now().astimezone()
sending_date.timestamp()
sending_date.strftime('%Y-%m-%d %H:%M:%S.%f')

'01945034-850a-45c1-8b7a-477ae7cb6b52' == b'01945034-850a-45c1-8b7a-477ae7cb6b52'.decode('utf-8')
import celery



a = ['Hi there Please Gime more infofr information *', 'Hi there Please Gime more infofr information *', 'Thanks!!', 'Why  you paid 10 usd? is correct?', 'Why  you paid 10 usd? is correct?', 'help', 'wow nice', 'super duper', 'super duper', 'omg!!!', 'ok i got it', 'toretto', 'a b c d  e f g h ', '1 2 3 4 5 6']
a[:-1]

['Hi there Please Gime more infofr information *',
 'Hi there Please Gime more infofr information *',
 'Thanks!!',
 'Why  you paid 10 usd? is correct?',
 'Why  you paid 10 usd? is correct?',
 'help',
 'wow nice',
 'super duper',
 'super duper',
 'omg!!!',
 'ok i got it',
 'toretto',
 'a b c d  e f g h ']

In [64]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig

# Load the tokenizer and model
cefr_grammar_tokenizer = AutoTokenizer.from_pretrained("hafidikhsan/distilbert-base-uncased-english-cefr-lexical-evaluation-dt-v1")
cefr_grammar_model = AutoModelForSequenceClassification.from_pretrained("hafidikhsan/distilbert-base-uncased-english-cefr-lexical-evaluation-dt-v1")
print("⚡️ Model [distilbert-base-uncased-english-cefr] loaded.")


def validate_(text: str):
   # Tokenize the input text
    inputs = cefr_grammar_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    
    # Extract input IDs and attention mask from tokenization output
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    # Perform inference
    with torch.no_grad():
        logits = cefr_grammar_model(input_ids=input_ids, attention_mask=attention_mask).logits


    # Apply softmax to get probabilities
    probabilities = F.softmax(logits, dim=1)

    # Convert probabilities to a list
    probabilities_list = probabilities.squeeze().tolist()
    probabilities_list = {cefr_grammar_model.config.id2label[i]: p for i, p in enumerate(probabilities_list)}
    p_max = max(probabilities_list, key=lambda x: probabilities_list[x])

    return probabilities_list, p_max


text = """If someone is upset on WhatsApp, you can be kind and listen to them. You can say sorry and try to help them feel better. Maybe you can ask what they need and find a way to fix the problem. It's good to stay calm and not be angry. You can offer a solution to make them happy again. Remember to be polite and show you care about their feelings. This can help to keep things nice and make them feel better. Being nice and helpful is a good way to handle tough times on WhatsApp.
"""




proba, p_max = validate_(text=text)

proba


⚡️ Model [distilbert-base-uncased-english-cefr] loaded.


{'A1': 0.562121570110321,
 'A2': 0.40971943736076355,
 'B1': 0.01917673647403717,
 'B2': 0.006161457393318415,
 'C1': 0.001137839164584875,
 'C2': 0.0016829922096803784}

In [ ]:
import requests

from dotenv import dotenv_values
from rich import print
secret = dotenv_values('../../.secret')

SPEECHDACE_API_KEY = secret['SPEECHDACE_API_KEY']


url = f"https://api.speechace.co/api/scoring/text/v9/json?key={SPEECHDACE_API_KEY}&dialect=en-us"

payload={'text': 'Dogs are sitting by the door.',
'question_info': '\'u1/q3\'',
'no_mc': '1'}
files=[
  ('user_audio_file',('03-01-08-01-02-01-04.wav',open('/Users/beltre.wilton/apps/ftlab_w2v2_ser/rawdata/ravdess_16k/03-01-08-01-02-01-04.wav','rb'),'audio/wav'))
]
headers = {}

response = requests.request("POST", url, headers=headers, data=payload, files=files)

# 'u1/q1'

# A unique identifier for the activity or question this user audio is answering.

# Structure this field to include as much info as possible to aid in reporting and analytics.

# For example: question_info='u1/q1' where:

# u1: means the question belongs to Unit 1 in your content
# q1: means this is question 1 within the unit
# You can add more levels as needed.

# Ensure no personally identifiable information is passed in this field.




In [74]:

from rich import print_json

print_json (response.text, highlight=False)




{
  "status": "success",
  "quota_remaining": -1,
  "text_score": {
    "text": "Dogs are sitting by the door.",
    "word_score_list": [
      {
        "word": "Dogs",
        "quality_score": 87,
        "phone_score_list": [
          {
            "phone": "d",
            "stress_level": null,
            "extent": [
              97,
              106
            ],
            "quality_score": 90,
            "word_extent": [
              0,
              1
            ],
            "sound_most_like": "d"
          },
          {
            "phone": "ao",
            "stress_level": 1,
            "extent": [
              106,
              121
            ],
            "quality_score": 59,
            "stress_score": 100,
            "predicted_stress_level": 0,
            "word_extent": [
              1,
              2
            ],
            "sound_most_like": "ah",
            "child_phones": [
              {
                "extent": [
                  106,
                  115
                ],
                "quality_score": 75,
                "sound_most_like": "aa"
              },
              {
                "extent": [
                  115,
                  121
                ],
                "quality_score": 35,
                "sound_most_like": "ah"
              }
            ]
          },
          {
            "phone": "g",
            "stress_level": null,
            "extent": [
              121,
              127
            ],
            "quality_score": 100,
            "word_extent": [
              2,
              3
            ],
            "sound_most_like": "g"
          },
          {
            "phone": "z",
            "stress_level": null,
            "extent": [
              127,
              136
            ],
            "quality_score": 99.33333333333333,
            "word_extent": [
              3,
              4
            ],
            "sound_most_like": "z"
          }
        ],
        "syllable_score_list": [
          {
            "phone_count": 4,
            "stress_level": 1,
            "letters": "dogs",
            "quality_score": 87,
            "stress_score": 100,
            "predicted_stress_level": 0,
            "extent": [
              97,
              136
            ]
          }
        ]
      },
      {
        "word": "are",
        "quality_score": 100,
        "phone_score_list": [
          {
            "phone": "er",
            "stress_level": 0,
            "extent": [
              136,
              145
            ],
            "quality_score": 99.66666666666667,
            "stress_score": 100,
            "predicted_stress_level": 0,
            "word_extent": [
              0,
              2
            ],
            "sound_most_like": "er"
          }
        ],
        "syllable_score_list": [
          {
            "phone_count": 1,
            "stress_level": 0,
            "letters": "are",
            "quality_score": 100,
            "stress_score": 100,
            "predicted_stress_level": 0,
            "extent": [
              136,
              145
            ]
          }
        ]
      },
      {
        "word": "sitting",
        "quality_score": 100,
        "phone_score_list": [
          {
            "phone": "s",
            "stress_level": null,
            "extent": [
              145,
              172
            ],
            "quality_score": 100,
            "word_extent": [
              0,
              1
            ],
            "sound_most_like": "s"
          },
          {
            "phone": "ih",
            "stress_level": 1,
            "extent": [
              172,
              178
            ],
            "quality_score": 100,
            "stress_score": 100,
            "predicted_stress_level": 1,
            "word_extent": [
              1,
              2
            ],
            "sound_most_like": "ih"
          },
          {
            "

In [77]:
r = response.json()

In [89]:
r['text_score']['word_score_list']

[{'word': 'Dogs',
  'quality_score': 87,
  'phone_score_list': [{'phone': 'd',
    'stress_level': None,
    'extent': [97, 106],
    'quality_score': 90,
    'word_extent': [0, 1],
    'sound_most_like': 'd'},
   {'phone': 'ao',
    'stress_level': 1,
    'extent': [106, 121],
    'quality_score': 59,
    'stress_score': 100,
    'predicted_stress_level': 0,
    'word_extent': [1, 2],
    'sound_most_like': 'ah',
    'child_phones': [{'extent': [106, 115],
      'quality_score': 75,
      'sound_most_like': 'aa'},
     {'extent': [115, 121], 'quality_score': 35, 'sound_most_like': 'ah'}]},
   {'phone': 'g',
    'stress_level': None,
    'extent': [121, 127],
    'quality_score': 100,
    'word_extent': [2, 3],
    'sound_most_like': 'g'},
   {'phone': 'z',
    'stress_level': None,
    'extent': [127, 136],
    'quality_score': 99.33333333333333,
    'word_extent': [3, 4],
    'sound_most_like': 'z'}],
  'syllable_score_list': [{'phone_count': 4,
    'stress_level': 1,
    'letters': 

In [ ]:
import dspy
from dsp.templates import passages2text

class SomeSignature(dspy.Signature):
    """Answer questions with short factoid answers and friendly, use emoji. Answer should be in the context."""
    chat_history: list[str] = dspy.InputField(format=passages2text, desc="Must consider relevant facts in the history of conversation.")
    question: str = dspy.InputField(desc="User question to be answered")
    answer: str = dspy.OutputField(desc="Contains the ai message output, often between 6 and 12 words")


class SomeModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predict = dspy.ChainOfThought(SomeSignature)
        
    def forward(self, question: str, chat_history: list[str]) -> dspy.Prediction:
        response = self.predict(chat_history=chat_history, question=question)
        return response


module = SomeModule()
chat_history = [
    "user: I there, my name is Rasputin",
    "ai: Hello, .... blab bla bla"
]
module(question="Do you remember my name?", chat_history=chat_history)

In [139]:

import random

basic_form = [
    "Thanks for filling out the basic form 🙌🏼 Now, please proceed to the assessment below ⤵️",
    "We appreciate you completing the basic form 🙌🏼 Your next step is to fill in the assessment below ⤵️",
    "Grateful for your completion of the basic form 🙌🏼 The next task is to tackle the assessment below ⤵️",
    "Thank you for finishing the basic form 🙌🏼 Up next is the assessment, please fill it out below ⤵️",
    "Thanks for completing the basic form 🙌🏼 The next thing to do is fill out the assessment below ⤵️",
    "We thank you for filling in the basic form 🙌🏼 Now, let's move on to the assessment below ⤵️",
    "Your completion of the basic form is appreciated 🙌🏼 Next, please complete the assessment below ⤵️",
    "Thank you for submitting the basic form 🙌🏼 The following step is to fill out the assessment below ⤵️",
    "Cheers for finishing the basic form 🙌🏼 Please proceed by filling out the assessment below ⤵️",
    "We appreciate you finishing the basic form 🙌🏼 Now, kindly complete the assessment below ⤵️"
]

assesment_form = [
    "Hey, many thanks! You've made great progress—well done! 🚀 The last step is to send me a voice note 🗣️ of up to 2 minutes for evaluation.",
    "Thank you so much! You've advanced significantly—congratulations! 🚀 The final task is to submit a voice note 🗣️ that's no longer than 2 minutes for review.",
    "Appreciate it! You've taken a big step forward—kudos! 🚀 Now, just send me a voice note 🗣️ under 2 minutes for evaluation.",
    "Thanks a lot! You've made an important leap—awesome job! 🚀 The last part is to send a voice note 🗣️ of no more than 2 minutes for assessment.",
    "Thank you very much! You've moved ahead impressively—great job! 🚀 The concluding step is to record and send a voice note 🗣️ under 2 minutes for evaluation.",
    "Hey, thanks a ton! You've achieved a major milestone—congrats! 🚀 The final thing is to send over a voice note 🗣️ up to 2 minutes long for evaluation purposes.",
    "Much appreciated! You've made remarkable progress—congratulations! 🚀 Please send a voice note 🗣️ lasting no longer than 2 minutes for the final review.",
    "Thanks a bunch! You've taken a crucial step forward—nice work! 🚀 The last action is to submit a voice note 🗣️ of 2 minutes or less for evaluation.",
    "Thank you immensely! You've made significant strides—well done! 🚀 The final requirement is to send me a voice note 🗣️ that’s no longer than 2 minutes for assessment.",
    "Hey, big thanks! You've moved forward significantly—excellent! 🚀 The last step is to record a voice note 🗣️ of up to 2 minutes for evaluation purposes."
]


voice_note = [
    "We got your voice note! ✅ Thanks so much! You've made excellent progress—great job! 🚀 The process is now complete.",
    "Your voice note has arrived! ✅ Many thanks! You've done fantastic work—congratulations! 🚀 We've successfully finished the process.",
    "Voice note received! ✅ Thank you very much! You've achieved great progress—well done! 🚀 The process is now finalized.",
    "We've got your voice note! ✅ Thanks a ton! You've advanced impressively—awesome job! 🚀 The process is now complete.",
    "Your voice note has been successfully received! ✅ Thanks a bunch! You've made amazing progress—congrats! 🚀 The process is now finished.",
    "Voice note received successfully! ✅ Thank you so much! You've reached a significant milestone—brilliant work! 🚀 The process is complete.",
    "We've received your voice note! ✅ Many thanks! You've progressed wonderfully—great job! 🚀 The process is now concluded.",
    "Your voice note is in! ✅ Thanks a lot! You've moved forward tremendously—excellent work! 🚀 The process is now done.",
    "Voice note has been received! ✅ Big thanks! You've accomplished great strides—well done! 🚀 We've completed the process successfully.",
    "Got your voice note! ✅ Thank you immensely! You've made substantial progress—fantastic job! 🚀 The process is now wrapped up."
]

voice_note_received_yet = [
    "We've received your voice note and it's now under validation! 🎧",
    "Your voice note has arrived and is being verified! 🎧",
    "Your voice note is in and currently going through validation! 🎧",
    "We've got your voice note and it's being checked! 🎧",
    "Your voice note has been received and is now under review! 🎧"
]


switch_to_text = [
    "Kindly use text communication for now. Thanks! 💬",
    "Please shift to text communication for the moment. Much appreciated! 💬",
    "Switch to text messages for the time being, please. Thanks a lot! 💬",
    "For now, please use text to communicate. Appreciate it! 💬",
    "We'd appreciate it if you could use text communication for now. Thanks! 💬",
    "Please communicate via text for the time being. Thanks! 💬",
    "Text communication is preferred for now. Thank you! 💬",
    "Could you please switch to text communication for the time being? Thanks! 💬",
    "For the moment, please switch to text messages. Appreciate it! 💬",
    "Please use text communication temporarily. Thank you! 💬"
]


assignment_reminder = [
    "Remember to fill out the form - it's quick and easy! ⏱️",
    "Take a brief moment to complete the form, it's just a minute or two! 📝",
    "Don't miss this step! Fill out the form now, it's fast and simple 😊",
    "Complete the form in just a flash - it only takes 1-2 minutes ⏱️",
    "A minute or two is all it takes - fill out the form now! 📝",
    "Don't forget this important step: complete the form today! 📝",
    "Spend a minute or two to fill out the form - it's easy peasy! 😊",
    "Take a short break to complete the form - it's quick and painless! ⏱️",
    "Fill out the form in no time - it's a breeze! ⏱️",
    "Complete the form now and be done in just a minute or two! 📝"
]


friendly_reminder = [
    "Don't forget to wrap up the evaluation ⏰—you're almost there! 💼",
    "A gentle nudge to finish the assessment 🌈—your dream job awaits! 🎉",
    "Remember to finalize the evaluation checklist 📝—hiring is just around the corner! 👍",
    "Complete the evaluation and take one giant leap towards your new role 🚀!",
    "Hey, don't miss this step! Finish the evaluation to land your ideal job 💻",
    "A heads up to tie up the evaluation loose ends 🎀—you're almost hired! 😊",
    "Get ready to celebrate! Finish the evaluation to move forward with your hiring 🎉",
    "Take the final step towards your new adventure: complete the evaluation today 🌟!",
    "Wrap up the evaluation and get one step closer to joining our team 👫!",
    "Finish strong! Complete the evaluation to seal the deal on your new job 📈",
    "Your new career is within reach! Don't forget to finish the evaluation 🌱"
]

voice_note_reminder = [
    "a final reminder: don't skip the voice note! ⏰ it should only take 2 minutes. thanks! 😊",
    "almost there! just one more thing: record a quick 2-minute voice note, please. 👍",
    "don't miss the last step: leave a voice note (it's short, we promise!) 📝",
    "remember to add your voice note - it's the final hurdle! 🏁 (max 2 minutes, thanks) 👊",
    "last but not least, please record a voice note - just 2 minutes, we appreciate it! 😊",
    "a friendly reminder: voice notes are essential! please add yours (takes just 2 minutes). 📞",
    "you're nearly done! just add a brief voice note (under 2 minutes) and you're all set. 👌",
    "don't forget to leave a voice message - it's the last step, and it's quick! ⏱️",
    "almost finished! please take 2 minutes to record a voice note - thanks for your time! 🙏",
    "final check: have you added your voice note? it's a quick 2-minute task, we promise! 💯"
]


def random_message(m_list: list):
    l = len(m_list) - 1
    r = random.randint(0, l)
    return m_list[r]

random_message(assignment_reminder)


"Don't miss this step! Fill out the form now, it's fast and simple 😊"